# Diamonds Data
Just to prove that diamonds are forever, we are going to revisit the diamonds data set.  This homework assignment will use the diamonds dataset to explore random forest decision tree models.

The diamonds.csv data set contains 10 columns:
- carat: Carat weight of the diamond
- cut: Describes cut quality of the diamond. Quality in increasing order Fair, Good, Very Good, Premium, Ideal
- color: Color of the diamond, with D being the best and J the worst
- clarity: How obvious inclusions are within the diamond:(in order from best to worst, FL = flawless, I3= level 3 inclusions) FL,IF, VVS1, etc.  See this web site for an exhaustive ranking of [clarity](https://4cs.gia.edu/en-us/diamond-clarity/?gclid=Cj0KCQjwnqH7BRDdARIsACTSAduMoc2KQbXkO94BxCfBNC5X8YyjAYcFpWThKQMW46cQj_3p0pZ0o84aAuagEALw_wcB).  The web site has a nice sliding scale you can drag to see the relationship between clarity grades.
- depth: depth % - The height of a diamond, measured from the culet to the table, divided by its average girdle diameter
- table: table% -  The width of the diamond's table expressed as a percentage of its average diameter
- price: The price of the diamond
- x: Length (mm)
- y: Width (mm)
- z: Height (mm)

In [1]:
# Grading Cell
enable_grid_search = False

The following cell is used to read the diamonds data set into the colab environment.  Do not change or modify the following cell.

In [2]:
%%bash
# Do not change or modify this file
# Need to install pyspark
# if pyspark is already installed, will print a message indicating pyspark already isntalled
pip install pyspark

# Download the data files from github
# If the data file does not exist in the colab environment
if [[ ! -f ./quotes_by_char.csv ]]; then 
   # download the data file from github and save it in this colab environment instance
   wget https://raw.githubusercontent.com/wewilli1/ist718_data/master/diamonds.csv  
fi

  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=90297645d33710cc3a4dd5ae66aa42f5ebf60502598ef7d64f35f08000d632a7
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


--2020-11-22 17:39:27--  https://raw.githubusercontent.com/wewilli1/ist718_data/master/diamonds.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3192560 (3.0M) [text/plain]
Saving to: ‘diamonds.csv’

     0K .......... .......... .......... .......... ..........  1% 4.05M 1s
    50K .......... .......... .......... .......... ..........  3% 10.9M 0s
   100K .......... .......... .......... .......... ..........  4% 4.88M 1s
   150K .......... .......... .......... .......... ..........  6% 19.5M 0s
   200K .......... .......... .......... .......... ..........  8% 5.73M 0s
   250K .......... .......... .......... .......... ..........  9% 32.4M 0s
   300K .......... .......... .......... .......... .......... 11% 36.3M 0s
   350K .......... .......... ........

# Question 1 (10 pts)
Read the diamonds.csv file into a spark data frame named `diamonds_df`.  Perform feature engineering as needed for training decision trees.  Name the new data frame diamonds_df_xformed.

In [3]:
# Do not delete or change this cell

# grading import statements
%matplotlib inline
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
import os

# Define a function to determine if we are running on data bricks
# Return true if running in the data bricks environment, false otherwise
def is_databricks():
    # get the databricks runtime version
    db_env = os.getenv("DATABRICKS_RUNTIME_VERSION")
    
    # if running on data bricks
    if db_env != None:
        return True
    else:
        return False

# Define a function to read the data file.  The full path data file name is constructed
# by checking runtime environment variables to determine if the runtime environment is 
# databricks, or a student's personal computer.  The full path file name is then
# constructed based on the runtime env.
# 
# Params
#   data_file_name: The base name of the data file to load
# 
# Returns the full path file name based on the runtime env
#
# Correct Usage Example (pass ONLY the full file name):
#   file_name_to_load = get_training_filename("sms_spam.csv") # correct - pass ONLY the full file name  
#   
# Incorrect Usage Example
#   file_name_to_load = get_training_filename("/sms_spam.csv") # incorrect - pass ONLY the full file name
#   file_name_to_load = get_training_filename("sms_spam.csv/") # incorrect - pass ONLY the full file name
#   file_name_to_load = get_training_filename("c:/users/will/data/sms_spam.csv") incorrect -pass ONLY the full file name
def get_training_filename(data_file_name):    
    # if running on data bricks
    if is_databricks():
        # build the full path file name assuming data brick env
        full_path_name = "dbfs:/FileStore/tables/%s" % data_file_name
    # else the data is assumed to be in the same dir as this notebook
    else:
        # Assume the student is running on their own computer and load the data
        # file from the same dir as this notebook
        full_path_name = data_file_name
    
    # return the full path file name to the caller
    return full_path_name

In [4]:
# your code here
from pyspark.ml import feature
from pyspark.ml import classification
from pyspark.sql import functions as fn
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexerModel
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
import os
#cut: Describes cut quality of the diamond. Quality in increasing order Fair, Good, Very Good, Premium, Ideal
#color: Color of the diamond. Quality in decreasing order 'D' 'E' 'F' 'G' 'H' 'I' 'J'
#clarity: Clarity of the diamond, Quality in increasing order 'I1''SI2''SI1''VS2''VS1''VVS2''VVS1''IF'

diamonds_df = spark.read.format("csv").option("header", "true").load(get_training_filename("diamonds.csv"))
diamonds_df = diamonds_df.drop('_c0')

si_cut = StringIndexerModel.from_labels(['Fair', 'Good', 'Very Good','Premium','Ideal'],
                                    inputCol="cut",
                                    outputCol="cut_idx")
si_color = StringIndexerModel.from_labels(['J','I','H','G','F','E','D'],
                                    inputCol="color",
                                    outputCol="color_idx")
si_clarity = StringIndexerModel.from_labels(['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF'],
                                    inputCol="clarity",
                                    outputCol="clarity_idx")

# Pipeline 
feature_engineering_pipe = Pipeline(stages=[si_cut,si_color,si_clarity])

#Fit the pipeline to dataset documents. 
model = feature_engineering_pipe.fit(diamonds_df)

# Feature Engineering
diamonds_df_xformed = model.transform(diamonds_df)

diamonds_df_xformed = diamonds_df_xformed.drop('cut','color','clarity')
diamonds_df_xformed = diamonds_df_xformed.withColumnRenamed("cut_idx","cut") \
    .withColumnRenamed("color_idx","color").withColumnRenamed("clarity_idx","clarity")

# check if there is any null or nan in dataframe
diamonds_df_xformed.select([count(when(isnull(c), c)).alias(c) for c in diamonds_df_xformed.columns]).show()
diamonds_df_xformed.select([count(when(isnan(c), c)).alias(c) for c in diamonds_df_xformed.columns]).show()

# Cast all dtypes from string to float in order to feed into LR model 
for col in diamonds_df_xformed.columns:
    diamonds_df_xformed = diamonds_df_xformed.withColumn(col, fn.col(col).cast("float"))


display(diamonds_df_xformed.toPandas().head())

+-----+-----+-----+-----+---+---+---+---+-----+-------+
|carat|depth|table|price|  x|  y|  z|cut|color|clarity|
+-----+-----+-----+-----+---+---+---+---+-----+-------+
|    0|    0|    0|    0|  0|  0|  0|  0|    0|      0|
+-----+-----+-----+-----+---+---+---+---+-----+-------+

+-----+-----+-----+-----+---+---+---+---+-----+-------+
|carat|depth|table|price|  x|  y|  z|cut|color|clarity|
+-----+-----+-----+-----+---+---+---+---+-----+-------+
|    0|    0|    0|    0|  0|  0|  0|  0|    0|      0|
+-----+-----+-----+-----+---+---+---+---+-----+-------+



,carat,depth,table,price,x,y,z,cut,color,clarity
0,0.23,61.500000,55.0,326.0,3.95,3.98,2.43,4.0,5.0,1.0
1,0.21,59.799999,61.0,326.0,3.89,3.84,2.31,3.0,5.0,2.0
2,0.23,56.900002,65.0,327.0,4.05,4.07,2.31,1.0,5.0,4.0
3,0.29,62.400002,58.0,334.0,4.20,4.23,2.63,3.0,1.0,3.0
4,0.31,63.299999,58.0,335.0,4.34,4.35,2.75,1.0,0.0,1.0


In [5]:
# Grading Cell - do not modify
display(diamonds_df_xformed.toPandas().head())

,carat,depth,table,price,x,y,z,cut,color,clarity
0,0.23,61.500000,55.0,326.0,3.95,3.98,2.43,4.0,5.0,1.0
1,0.21,59.799999,61.0,326.0,3.89,3.84,2.31,3.0,5.0,2.0
2,0.23,56.900002,65.0,327.0,4.05,4.07,2.31,1.0,5.0,4.0
3,0.29,62.400002,58.0,334.0,4.20,4.23,2.63,3.0,1.0,3.0
4,0.31,63.299999,58.0,335.0,4.34,4.35,2.75,1.0,0.0,1.0


##### Grading Feedback Cell

The following questions will create a random forest regressor model, train the model using a grid search, and use the model for inference.  The goal is to see if we can improve upon the linear regression score from homework 3. You can find the spark documentation for the random forest regressor [here](https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-regression).

# Question 2 (20 pts)
Create and train your random forest regressor model using a grid search in the cell below.  You are free to use K-Fold Cross validation if you wish.  Your grid search must be entirely encapsulated in the `if enable_grid_search` if statement.  The `enable_grid_search` Boolean is defined in a grading cell above.  You will disable the grid search before you submit by setting enable_grid_search to false.  Setting enable_grid_search to false should not result in a runtime error.  You will not receive full credit if any part of your grid search is outside of the if statement or if runtime errros result from setting the `enable_grid_search` variable to false.

In [6]:
# your code here
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator


if enable_grid_search:
  diamonds_df_xformed = diamonds_df_xformed.select([diamonds_df_xformed.columns[3]] + diamonds_df_xformed.columns[:3] + diamonds_df_xformed.columns[4:])
  training_df, testing_df = diamonds_df_xformed.randomSplit([0.7, 0.3])

  # Establish pipeline object
  va = VectorAssembler().setInputCols(training_df.columns[2:]).setOutputCol('features')
  fi = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4)
  rf = RandomForestRegressor(featuresCol = 'indexedFeatures', labelCol="price")

  rmse_evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")

  # Create search grid for training
  paramGrid = ParamGridBuilder().addGrid(rf.maxDepth, [8, 9, 10]).addGrid(rf.maxBins, [20, 23,25]).addGrid(rf.numTrees, [40,50,60]).build()
  cv = CrossValidator(estimator=Pipeline(stages=[va, fi, rf]), estimatorParamMaps=paramGrid, evaluator=rmse_evaluator, numFolds=3)

  cv_fit = cv.fit(training_df)
  cv_prediction = cv_fit.transform(testing_df)
  cv_rmse = rmse_evaluator.evaluate(cv_prediction)
  print("RMSE is:", cv_rmse)

  grid_param_dict = cv_fit.bestModel.stages[-1].extractParamMap()
  
  result = {}
  for x,y in grid_param_dict.items():
    result[x.name]= y

  best_maxdepth = result["maxDepth"]
  best_maxbins = result["maxBins"]
  best_numTrees = result["numTrees"]

  print('Best Param (maxdepth):{:.3f}'.format(best_maxdepth))
  print('Best Param (maxbins):{:.3f}'.format(best_maxbins))
  print('Best Param (numTrees):{:.3f}'.format(best_numTrees))


  pass

# Question 3 (20 pts)
Create a pipeline named `best_pipe` that hard codes the tuning parameters from the best model found by the grid search in question 2 above.  Train and test best_pipe.  Do not use k-fold cross validation in question 3.  Clearly print the resulting train and test MSE for best_pipe so it's easy for the graders to see your resulting MSEs.

In [7]:
# Your code here
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator

training_df, testing_df = diamonds_df_xformed.randomSplit([0.7, 0.3])

va = VectorAssembler().setInputCols(training_df.columns[1:]).setOutputCol('features')
fi = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4)
rf = RandomForestRegressor().setFeaturesCol('indexedFeatures').setLabelCol("price").setMaxDepth(10).setNumTrees(40).setMaxBins(25)
mse_evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mse")

best_pipe = Pipeline(stages=[va, fi, rf]).fit(training_df)
cv_train_predict = best_pipe.transform(training_df)
cv_test_predict = best_pipe.transform(testing_df)

cv_train_mse = mse_evaluator.evaluate(cv_train_predict)
cv_test_mse = mse_evaluator.evaluate(cv_test_predict)

print("Training MSE is:", cv_train_mse)
print("Testing MSE is:", cv_test_mse)


Training MSE is: 104150.7949287362
Testing MSE is: 126581.30101988133


# Question 4 (20 pts)
Use your best_pipe pipeline in question 3 for inference.  Create a pandas data frame named `rf_feature_importance` which contains 2 columns: `feature`, and `importance`.  Load the feature column with the feature name and the importance column with the feature importance score as determined by the random forest model. Sort the feature importances from high to low such that the most important feature is in the first row of the data frame.

In [8]:
# your code here
rf_feature_importance = pd.DataFrame(list(zip(diamonds_df_xformed.columns[1:], best_pipe.stages[-1].featureImportances.toArray())), columns = ['feature', 'importance']).sort_values('importance',ascending=False)

In [9]:
# grading cell - do not modify
display(rf_feature_importance)

,feature,importance
2,price,0.423026
4,y,0.258660
3,x,0.178683
5,z,0.112483
8,clarity,0.015754
7,color,0.006607
1,table,0.003103
0,depth,0.001032
6,cut,0.000652


# Question 5 (20 pts)
Write code to print the decision logic for any of the trees in the forest from the best_pipe pipeline.  Copy the printed decision text to the tree printout markdown cell below and retain the same formatting and indentation as the code printout so it's easy for the graders to view the data.  You need to double click the "Your Decision Tree Print Out Here" markdown cell and paste your output inside the two sets of triple quotes. The triple quotes are jupyter markdown indicating you want to present code.  Essentially, replace the text inside the triple quotes with your tree printout.  Solutions that do not maintain readable formatting will not receive full credit.

Add comments to the markdown cell below describing how the root node is split:  Describe 2 things in the markdown cell.  1) What specific predictor variable is being split and what is the value that determines the left / right split.  2) We need you to paste the tree decision logic output from your run in the markdown cell because the top level split may change from run to run.  If the graders run your notebook, the top level split for the tree may be different than the top level split from when you made the run.  Describe why the top level predictor changes from run to run.


In [10]:
# your code here
print(best_pipe.stages[-1].trees[0].toDebugString)

DecisionTreeRegressionModel: uid=dtr_fea63d367f7c, depth=10, numNodes=1669, numFeatures=9
  If (feature 4 <= 6.365000009536743)
   If (feature 3 <= 5.634999990463257)
    If (feature 2 <= 1356.5)
     If (feature 4 <= 4.684999942779541)
      If (feature 2 <= 740.5)
       If (feature 2 <= 595.5)
        If (feature 2 <= 501.5)
         If (feature 8 <= 4.5)
          If (feature 3 <= 4.244999885559082)
           If (feature 8 <= 2.5)
            Predict: 397.85185185185185
           Else (feature 8 > 2.5)
            Predict: 431.86475409836066
          Else (feature 3 > 4.244999885559082)
           If (feature 5 <= 2.784999966621399)
            Predict: 445.97301854974705
           Else (feature 5 > 2.784999966621399)
            Predict: 459.82456140350877
         Else (feature 8 > 4.5)
          If (feature 3 <= 4.315000057220459)
           If (feature 7 <= 3.5)
            Predict: 453.05479452054794
           Else (feature 7 > 3.5)
            Predict: 480.15277777777777

 ```
DecisionTreeRegressionModel: uid=dtr_34c26ef3c78a, depth=10, numNodes=1605, numFeatures=9
  If (feature 4 <= 6.375)
   If (feature 4 <= 5.615000009536743)
    If (feature 1 <= 1330.5)
     If (feature 4 <= 4.674999952316284)
      If (feature 8 <= 5.5)
       If (feature 2 <= 0.32500000298023224)
        If (feature 7 <= 2.5)
         If (feature 1 <= 590.5)
          If (feature 8 <= 2.5)
           If (feature 8 <= 1.5)
            Predict: 402.1111111111111
           Else (feature 8 > 1.5)
            Predict: 460.57706093189967
          Else (feature 8 > 2.5)
           If (feature 3 <= 4.255000114440918)
            Predict: 456.4307692307692
           Else (feature 3 > 4.255000114440918)
            Predict: 497.4713541666667
         Else (feature 1 > 590.5)
          If (feature 1 <= 670.5)
           If (feature 5 <= 2.6750000715255737)
            Predict: 616.5507246376811
           Else (feature 5 > 2.6750000715255737)
            Predict: 626.1944444444445
          Else (feature 1 > 670.5)
           If (feature 1 <= 745.5)
            Predict: 698.2222222222222
           Else (feature 1 > 745.5)
            Predict: 788.4285714285714
        Else (feature 7 > 2.5)
         If (feature 6 <= 2.5)
          If (feature 3 <= 4.255000114440918)
           If (feature 8 <= 4.5)
            Predict: 473.62213740458014
           Else (feature 8 > 4.5)
            Predict: 565.6981132075472
          Else (feature 3 > 4.255000114440918)
           If (feature 8 <= 2.5)
            Predict: 547.176245210728
           Else (feature 8 > 2.5)
            Predict: 661.8029850746269
         Else (feature 6 > 2.5)
          If (feature 1 <= 745.5)
           If (feature 2 <= 0.29500000178813934)
            Predict: 527.2294372294373
           Else (feature 2 > 0.29500000178813934)
            Predict: 626.3862348178137
          Else (feature 1 > 745.5)
           If (feature 8 <= 2.5)
            Predict: 756.0
           Else (feature 8 > 2.5)
            Predict: 859.6285046728972
       Else (feature 2 > 0.32500000298023224)
        If (feature 1 <= 745.5)
         If (feature 2 <= 0.38499999046325684)
          If (feature 8 <= 2.5)
           If (feature 7 <= 4.5)
            Predict: 556.2203389830509
           Else (feature 7 > 4.5)
            Predict: 605.5709090909091
          Else (feature 8 > 2.5)
           If (feature 0 <= 53.95000076293945)
            Predict: 607.2
           Else (feature 0 > 53.95000076293945)
            Predict: 647.2555555555556
         Else (feature 2 > 0.38499999046325684)
          If (feature 8 <= 1.5)
           If (feature 7 <= 4.5)
            Predict: 630.1666666666666
           Else (feature 7 > 4.5)
            Predict: 580.5
          Else (feature 8 > 1.5)
           If (feature 3 <= 4.664999961853027)
            Predict: 652.1333333333333
           Else (feature 3 > 4.664999961853027)
            Predict: 714.0
        Else (feature 1 > 745.5)
         If (feature 2 <= 0.3449999988079071)
          If (feature 7 <= 3.5)
           If (feature 6 <= 3.5)
            Predict: 848.4615384615385
           Else (feature 6 > 3.5)
            Predict: 818.6491228070175
          Else (feature 7 > 3.5)
           If (feature 8 <= 2.5)
            Predict: 779.5641025641025
           Else (feature 8 > 2.5)
            Predict: 890.1493055555555
         Else (feature 2 > 0.3449999988079071)
          If (feature 3 <= 4.525000095367432)
           If (feature 6 <= 2.5)
            Predict: 783.4285714285714
           Else (feature 6 > 2.5)
            Predict: 851.3863636363636
          Else (feature 3 > 4.525000095367432)
           If (feature 1 <= 906.5)
            Predict: 824.8979591836735
           Else (feature 1 > 906.5)
            Predict: 1030.4440789473683
      Else (feature 8 > 5.5)
       If (feature 2 <= 0.29500000178813934)
        If (feature 6 <= 2.5)
         If (feature 4 <= 4.255000114440918)
          If (feature 0 <= 61.95000076293945)
           If (feature 1 <= 590.5)
            Predict: 507.1
           Else (feature 1 > 590.5)
            Predict: 650.28125
          Else (feature 0 > 61.95000076293945)
           If (feature 8 <= 6.5)
            Predict: 440.6666666666667
           Else (feature 8 > 6.5)
            Predict: 506.0
         Else (feature 4 > 4.255000114440918)
          If (feature 1 <= 670.5)
           If (feature 1 <= 590.5)
            Predict: 534.6666666666666
           Else (feature 1 > 590.5)
            Predict: 618.8333333333334
          Else (feature 1 > 670.5)
           Predict: 815.0
        Else (feature 6 > 2.5)
         If (feature 1 <= 670.5)
          If (feature 1 <= 500.5)
           If (feature 0 <= 59.10000038146973)
            Predict: 450.85714285714283
           Else (feature 0 > 59.10000038146973)
            Predict: 414.0
          Else (feature 1 > 500.5)
           If (feature 0 <= 57.04999923706055)
            Predict: 580.8478260869565
           Else (feature 0 > 57.04999923706055)
            Predict: 602.6071428571429
         Else (feature 1 > 670.5)
          If (feature 1 <= 745.5)
           If (feature 8 <= 6.5)
            Predict: 708.45
           Else (feature 8 > 6.5)
            Predict: 681.7777777777778
          Else (feature 1 > 745.5)
           If (feature 5 <= 2.625)
            Predict: 781.2962962962963
           Else (feature 5 > 2.625)
            Predict: 820.75
       Else (feature 2 > 0.29500000178813934)
        If (feature 4 <= 4.525000095367432)
         If (feature 1 <= 906.5)
          If (feature 1 <= 745.5)
           If (feature 4 <= 4.255000114440918)
            Predict: 719.9090909090909
           Else (feature 4 > 4.255000114440918)
            Predict: 663.2424242424242
          Else (feature 1 > 745.5)
           If (feature 0 <= 60.04999923706055)
            Predict: 837.9795918367347
           Else (feature 0 > 60.04999923706055)
            Predict: 788.9230769230769
         Else (feature 1 > 906.5)
          If (feature 1 <= 1005.5)
           If (feature 2 <= 0.32500000298023224)
            Predict: 941.3556701030927
           Else (feature 2 > 0.32500000298023224)
            Predict: 965.9722222222222
          Else (feature 1 > 1005.5)
           If (feature 1 <= 1119.5)
            Predict: 1050.010152284264
           Else (feature 1 > 1119.5)
            Predict: 1183.1311475409836
        Else (feature 4 > 4.525000095367432)
         If (feature 1 <= 1005.5)
          If (feature 7 <= 1.5)
           If (feature 3 <= 4.664999961853027)
            Predict: 713.2380952380952
           Else (feature 3 > 4.664999961853027)
            Predict: 921.0
          Else (feature 7 > 1.5)
           If (feature 5 <= 2.875)
            Predict: 895.7549019607843
           Else (feature 5 > 2.875)
            Predict: 952.6666666666666
         Else (feature 1 > 1005.5)
          If (feature 7 <= 3.5)
           If (feature 1 <= 1119.5)
            Predict: 1067.6410256410256
           Else (feature 1 > 1119.5)
            Predict: 1184.142857142857
          Else (feature 7 > 3.5)
           If (feature 1 <= 1119.5)
            Predict: 1077.25
           Else (feature 1 > 1119.5)
            Predict: 1231.8775510204082
     Else (feature 4 > 4.674999952316284)
      If (feature 5 <= 3.15500009059906)
       If (feature 1 <= 906.5)
        If (feature 1 <= 745.5)
         If (feature 5 <= 3.0049999952316284)
          If (feature 7 <= 0.5)
           If (feature 2 <= 0.38499999046325684)
            Predict: 551.875
           Else (feature 2 > 0.38499999046325684)
            Predict: 629.5079365079365
          Else (feature 7 > 0.5)
           If (feature 7 <= 2.5)
            Predict: 661.375939849624
           Else (feature 7 > 2.5)
            Predict: 681.2114285714285
         Else (feature 5 > 3.0049999952316284)
          If (feature 7 <= 4.5)
           If (feature 8 <= 1.5)
            Predict: 658.4615384615385
           Else (feature 8 > 1.5)
            Predict: 707.2380952380952
          Else (feature 7 > 4.5)
           If (feature 2 <= 0.4350000023841858)
            Predict: 675.5
           Else (feature 2 > 0.4350000023841858)
            Predict: 694.0
        Else (feature 1 > 745.5)
         If (feature 1 <= 826.5)
          If (feature 0 <= 54.95000076293945)
           If (feature 3 <= 4.664999961853027)
            Predict: 784.375
           Else (feature 3 > 4.664999961853027)
            Predict: 769.0
          Else (feature 0 > 54.95000076293945)
           If (feature 2 <= 0.4050000011920929)
            Predict: 790.0615384615385
           Else (feature 2 > 0.4050000011920929)
            Predict: 783.0
         Else (feature 1 > 826.5)
          If (feature 6 <= 3.5)
           If (feature 8 <= 6.5)
            Predict: 863.5074074074074
           Else (feature 8 > 6.5)
            Predict: 827.5
          Else (feature 6 > 3.5)
           If (feature 0 <= 56.04999923706055)
            Predict: 866.6062992125984
           Else (feature 0 > 56.04999923706055)
            Predict: 876.5666666666667
       Else (feature 1 > 906.5)
        If (feature 1 <= 1119.5)
         If (feature 3 <= 4.755000114440918)
          If (feature 7 <= 5.5)
           If (feature 6 <= 1.5)
            Predict: 968.3333333333334
           Else (feature 6 > 1.5)
            Predict: 1009.7625329815304
          Else (feature 7 > 5.5)
           If (feature 1 <= 1005.5)
            Predict: 959.1935483870968
           Else (feature 1 > 1005.5)
            Predict: 1072.7826086956522
         Else (feature 3 > 4.755000114440918)
          If (feature 0 <= 57.04999923706055)
           If (feature 1 <= 1005.5)
            Predict: 957.2227074235808
           Else (feature 1 > 1005.5)
            Predict: 1062.909420289855
          Else (feature 0 > 57.04999923706055)
           If (feature 1 <= 1005.5)
            Predict: 960.0
           Else (feature 1 > 1005.5)
            Predict: 1063.217741935484
        Else (feature 1 > 1119.5)
         If (feature 3 <= 4.865000009536743)
          If (feature 8 <= 4.5)
           If (feature 6 <= 3.5)
            Predict: 1151.219512195122
           Else (feature 6 > 3.5)
            Predict: 1188.283950617284
          Else (feature 8 > 4.5)
           If (feature 8 <= 6.5)
            Predict: 1204.8233215547702
           Else (feature 8 > 6.5)
            Predict: 1229.1538461538462
         Else (feature 3 > 4.865000009536743)
          If (feature 2 <= 0.4050000011920929)
           If (feature 5 <= 2.875)
            Predict: 1152.0
           Else (feature 5 > 2.875)
            Predict: 1186.0
          Else (feature 2 > 0.4050000011920929)
           If (feature 7 <= 2.5)
            Predict: 1189.7142857142858
           Else (feature 7 > 2.5)
            Predict: 1233.444
      Else (feature 5 > 3.15500009059906)
       If (feature 1 <= 1119.5)
        If (feature 1 <= 1005.5)
         If (feature 8 <= 0.5)
          If (feature 3 <= 5.105000019073486)
           If (feature 7 <= 4.5)
            Predict: 627.0
           Else (feature 7 > 4.5)
            Predict: 720.0
          Else (feature 3 > 5.105000019073486)
           If (feature 5 <= 3.4550000429153442)
            Predict: 889.0833333333334
           Else (feature 5 > 3.4550000429153442)
            Predict: 963.2
         Else (feature 8 > 0.5)
          If (feature 2 <= 0.5049999952316284)
           If (feature 1 <= 906.5)
            Predict: 842.8571428571429
           Else (feature 1 > 906.5)
            Predict: 959.8235294117648
          Else (feature 2 > 0.5049999952316284)
           If (feature 0 <= 56.95000076293945)
            Predict: 957.8518518518518
           Else (feature 0 > 56.95000076293945)
            Predict: 976.4782608695652
        Else (feature 1 > 1005.5)
         If (feature 5 <= 3.4550000429153442)
          If (feature 3 <= 5.335000038146973)
           If (feature 8 <= 1.5)
            Predict: 1080.9710144927535
           Else (feature 8 > 1.5)
            Predict: 1064.2164948453608
          Else (feature 3 > 5.335000038146973)
           Predict: 1050.4705882352941
         Else (feature 5 > 3.4550000429153442)
          If (feature 0 <= 55.04999923706055)
           If (feature 5 <= 3.7350000143051147)
            Predict: 1069.0
           Else (feature 5 > 3.7350000143051147)
            Predict: 1049.0
          Else (feature 0 > 55.04999923706055)
           If (feature 4 <= 5.3450000286102295)
            Predict: 1092.0
           Else (feature 4 > 5.3450000286102295)
            Predict: 1107.2857142857142
       Else (feature 1 > 1119.5)
        If (feature 7 <= 3.5)
         If (feature 6 <= 0.5)
          If (feature 3 <= 5.335000038146973)
           If (feature 0 <= 56.04999923706055)
            Predict: 1229.2
           Else (feature 0 > 56.04999923706055)
            Predict: 1170.0
          Else (feature 3 > 5.335000038146973)
           If (feature 3 <= 5.625)
            Predict: 1138.0
           Else (feature 3 > 5.625)
            Predict: 1174.6666666666667
         Else (feature 6 > 0.5)
          If (feature 2 <= 0.5049999952316284)
           If (feature 6 <= 1.5)
            Predict: 1284.5
           Else (feature 6 > 1.5)
            Predict: 1197.88
          Else (feature 2 > 0.5049999952316284)
           If (feature 0 <= 60.95000076293945)
            Predict: 1228.2857142857142
           Else (feature 0 > 60.95000076293945)
            Predict: 1180.75
        Else (feature 7 > 3.5)
         If (feature 6 <= 1.5)
          If (feature 3 <= 5.335000038146973)
           If (feature 0 <= 58.04999923706055)
            Predict: 1197.7857142857142
           Else (feature 0 > 58.04999923706055)
            Predict: 1227.2857142857142
          Else (feature 3 > 5.335000038146973)
           If (feature 2 <= 0.5649999976158142)
            Predict: 1176.0
           Else (feature 2 > 0.5649999976158142)
            Predict: 1234.6
         Else (feature 6 > 1.5)
          If (feature 7 <= 5.5)
           If (feature 8 <= 1.5)
            Predict: 1234.8095238095239
           Else (feature 8 > 1.5)
            Predict: 1285.7702702702702
          Else (feature 7 > 5.5)
           If (feature 5 <= 3.215000033378601)
            Predict: 1264.7586206896551
           Else (feature 5 > 3.215000033378601)
            Predict: 1284.2222222222222
    Else (feature 1 > 1330.5)
     If (feature 5 <= 2.625)
      If (feature 8 <= 1.5)
       Predict: 18034.0
      Else (feature 8 > 1.5)
       Predict: 15686.0
     Else (feature 5 > 2.625)
      If (feature 8 <= 4.5)
       If (feature 2 <= 0.6949999928474426)
        If (feature 4 <= 5.2149999141693115)
         If (feature 7 <= 5.5)
          If (feature 3 <= 5.105000019073486)
           If (feature 8 <= 2.5)
            Predict: 1407.9591836734694
           Else (feature 8 > 2.5)
            Predict: 1563.3296296296296
          Else (feature 3 > 5.105000019073486)
           If (feature 1 <= 1649.5)
            Predict: 1487.7371794871794
           Else (feature 1 > 1649.5)
            Predict: 1762.5608308605342
         Else (feature 7 > 5.5)
          If (feature 8 <= 2.5)
           If (feature 3 <= 5.204999923706055)
            Predict: 1534.6101694915253
           Else (feature 3 > 5.204999923706055)
            Predict: 1612.1052631578948
          Else (feature 8 > 2.5)
           If (feature 4 <= 5.105000019073486)
            Predict: 1684.9186046511627
           Else (feature 4 > 5.105000019073486)
            Predict: 1843.8015267175572
        Else (feature 4 > 5.2149999141693115)
         If (feature 7 <= 4.5)
          If (feature 8 <= 2.5)
           If (feature 1 <= 1649.5)
            Predict: 1446.2602040816328
           Else (feature 1 > 1649.5)
            Predict: 1778.325
          Else (feature 8 > 2.5)
           If (feature 7 <= 2.5)
            Predict: 1596.5320512820513
           Else (feature 7 > 2.5)
            Predict: 1823.986328125
         Else (feature 7 > 4.5)
          If (feature 3 <= 5.335000038146973)
           If (feature 1 <= 1873.0)
            Predict: 1660.4514285714286
           Else (feature 1 > 1873.0)
            Predict: 1997.8695652173913
          Else (feature 3 > 5.335000038146973)
           If (feature 8 <= 2.5)
            Predict: 1711.1986754966888
           Else (feature 8 > 2.5)
            Predict: 2088.8172588832485
       Else (feature 2 > 0.6949999928474426)
        If (feature 1 <= 3939.0)
         If (feature 1 <= 2184.5)
          If (feature 7 <= 0.5)
           If (feature 5 <= 3.59499990940094)
            Predict: 1729.25
           Else (feature 5 > 3.59499990940094)
            Predict: 1607.875
          Else (feature 7 > 0.5)
           If (feature 2 <= 0.7049999833106995)
            Predict: 1950.8333333333333
           Else (feature 2 > 0.7049999833106995)
            Predict: 1894.948717948718
         Else (feature 1 > 2184.5)
          If (feature 1 <= 2535.5)
           If (feature 7 <= 2.5)
            Predict: 2293.8695652173915
           Else (feature 7 > 2.5)
            Predict: 2367.529411764706
          Else (feature 1 > 2535.5)
           If (feature 1 <= 2903.5)
            Predict: 2672.8695652173915
           Else (feature 1 > 2903.5)
            Predict: 3116.0555555555557
        Else (feature 1 > 3939.0)
         If (feature 0 <= 56.04999923706055)
          Predict: 7188.0
         Else (feature 0 > 56.04999923706055)
          Predict: 6115.0
      Else (feature 8 > 4.5)
       If (feature 1 <= 2535.5)
        If (feature 8 <= 5.5)
         If (feature 3 <= 4.865000009536743)
          If (feature 6 <= 3.5)
           If (feature 6 <= 2.5)
            Predict: 1360.25
           Else (feature 6 > 2.5)
            Predict: 1356.0
          Else (feature 6 > 3.5)
           If (feature 5 <= 2.9450000524520874)
            Predict: 1381.0
           Else (feature 5 > 2.9450000524520874)
            Predict: 1396.6666666666667
         Else (feature 3 > 4.865000009536743)
          If (feature 5 <= 3.215000033378601)
           If (feature 2 <= 0.5049999952316284)
            Predict: 1888.0113636363637
           Else (feature 2 > 0.5049999952316284)
            Predict: 2003.8201058201057
          Else (feature 5 > 3.215000033378601)
           If (feature 7 <= 3.5)
            Predict: 1988.2251655629138
           Else (feature 7 > 3.5)
            Predict: 2339.8101265822784
        Else (feature 8 > 5.5)
         If (feature 1 <= 1873.0)
          If (feature 4 <= 5.105000019073486)
           If (feature 1 <= 1649.5)
            Predict: 1422.016304347826
           Else (feature 1 > 1649.5)
            Predict: 1775.2758620689656
          Else (feature 4 > 5.105000019073486)
           If (feature 1 <= 1649.5)
            Predict: 1544.095238095238
           Else (feature 1 > 1649.5)
            Predict: 1793.2285714285715
         Else (feature 1 > 1873.0)
          If (feature 3 <= 5.204999923706055)
           If (feature 7 <= 3.5)
            Predict: 2033.3253012048192
           Else (feature 7 > 3.5)
            Predict: 2244.7083333333335
          Else (feature 3 > 5.204999923706055)
           If (feature 5 <= 3.284999966621399)
            Predict: 2163.927536231884
           Else (feature 5 > 3.284999966621399)
            Predict: 2292.938775510204
       Else (feature 1 > 2535.5)
        If (feature 8 <= 5.5)
         If (feature 0 <= 61.95000076293945)
          If (feature 7 <= 5.5)
           If (feature 1 <= 2903.5)
            Predict: 2652.8070175438597
           Else (feature 1 > 2903.5)
            Predict: 3095.409090909091
          Else (feature 7 > 5.5)
           If (feature 1 <= 2903.5)
            Predict: 2703.25
           Else (feature 1 > 2903.5)
            Predict: 3314.0
         Else (feature 0 > 61.95000076293945)
          Predict: 4368.0
        Else (feature 8 > 5.5)
         If (feature 5 <= 3.284999966621399)
          If (feature 1 <= 3416.5)
           If (feature 1 <= 2903.5)
            Predict: 2727.285714285714
           Else (feature 1 > 2903.5)
            Predict: 3142.785714285714
          Else (feature 1 > 3416.5)
           If (feature 6 <= 2.5)
            Predict: 4440.0
           Else (feature 6 > 2.5)
            Predict: 3648.2727272727275
         Else (feature 5 > 3.284999966621399)
          If (feature 7 <= 5.5)
           If (feature 1 <= 2903.5)
            Predict: 2719.8571428571427
           Else (feature 1 > 2903.5)
            Predict: 3251.8333333333335
          Else (feature 7 > 5.5)
           If (feature 6 <= 2.5)
            Predict: 3912.714285714286
           Else (feature 6 > 2.5)
            Predict: 4300.5
   Else (feature 4 > 5.615000009536743)
    If (feature 1 <= 3939.0)
     If (feature 4 <= 6.055000066757202)
      If (feature 4 <= 5.8450000286102295)
       If (feature 1 <= 2535.5)
        If (feature 1 <= 2184.5)
         If (feature 8 <= 0.5)
          If (feature 6 <= 3.5)
           If (feature 7 <= 3.5)
            Predict: 1365.1666666666667
           Else (feature 7 > 3.5)
            Predict: 1142.3333333333333
          Else (feature 6 > 3.5)
           If (feature 1 <= 1873.0)
            Predict: 1761.0
           Else (feature 1 > 1873.0)
            Predict: 1928.0
         Else (feature 8 > 0.5)
          If (feature 7 <= 1.5)
           If (feature 1 <= 1873.0)
            Predict: 1705.6024096385543
           Else (feature 1 > 1873.0)
            Predict: 2024.6027397260275
          Else (feature 7 > 1.5)
           If (feature 6 <= 2.5)
            Predict: 2046.7926829268292
           Else (feature 6 > 2.5)
            Predict: 1988.16875
        Else (feature 1 > 2184.5)
         If (feature 6 <= 3.5)
          If (feature 8 <= 2.5)
           If (feature 5 <= 3.59499990940094)
            Predict: 2328.3254237288133
           Else (feature 5 > 3.59499990940094)
            Predict: 2358.830508474576
          Else (feature 8 > 2.5)
           If (feature 7 <= 1.5)
            Predict: 2353.5858585858587
           Else (feature 7 > 1.5)
            Predict: 2413.8098159509204
         Else (feature 6 > 3.5)
          If (feature 7 <= 3.5)
           If (feature 5 <= 3.5449999570846558)
            Predict: 2380.8953488372094
           Else (feature 5 > 3.5449999570846558)
            Predict: 2366.128712871287
          Else (feature 7 > 3.5)
           If (feature 7 <= 4.5)
            Predict: 2392.1428571428573
           Else (feature 7 > 4.5)
            Predict: 2413.3720930232557
       Else (feature 1 > 2535.5)
        If (feature 6 <= 3.5)
         If (feature 8 <= 3.5)
          If (feature 2 <= 0.7049999833106995)
           If (feature 1 <= 2903.5)
            Predict: 2707.222222222222
           Else (feature 1 > 2903.5)
            Predict: 3073.0
          Else (feature 2 > 0.7049999833106995)
           If (feature 7 <= 4.5)
            Predict: 2761.016
           Else (feature 7 > 4.5)
            Predict: 2889.81220657277
         Else (feature 8 > 3.5)
          If (feature 8 <= 5.5)
           If (feature 7 <= 3.5)
            Predict: 2778.867469879518
           Else (feature 7 > 3.5)
            Predict: 3156.3518518518517
          Else (feature 8 > 5.5)
           If (feature 1 <= 2903.5)
            Predict: 2690.277777777778
           Else (feature 1 > 2903.5)
            Predict: 3429.4754098360654
        Else (feature 6 > 3.5)
         If (feature 7 <= 2.5)
          If (feature 7 <= 1.5)
           If (feature 1 <= 2903.5)
            Predict: 2646.8125
           Else (feature 1 > 2903.5)
            Predict: 3024.133333333333
          Else (feature 7 > 1.5)
           If (feature 1 <= 2903.5)
            Predict: 2685.0285714285715
           Else (feature 1 > 2903.5)
            Predict: 3146.423076923077
         Else (feature 7 > 2.5)
          If (feature 8 <= 2.5)
           If (feature 8 <= 1.5)
            Predict: 2684.3555555555554
           Else (feature 8 > 1.5)
            Predict: 2870.268292682927
          Else (feature 8 > 2.5)
           If (feature 8 <= 4.5)
            Predict: 3162.176308539945
           Else (feature 8 > 4.5)
            Predict: 3457.6666666666665
      Else (feature 4 > 5.8450000286102295)
       If (feature 1 <= 2903.5)
        If (feature 8 <= 0.5)
         If (feature 5 <= 3.9950000047683716)
          If (feature 5 <= 3.5449999570846558)
           Predict: 1865.0
          Else (feature 5 > 3.5449999570846558)
           If (feature 7 <= 4.5)
            Predict: 2041.0
           Else (feature 7 > 4.5)
            Predict: 2357.8
         Else (feature 5 > 3.9950000047683716)
          If (feature 5 <= 4.125)
           Predict: 1791.3333333333333
          Else (feature 5 > 4.125)
           If (feature 7 <= 3.5)
            Predict: 1749.0
           Else (feature 7 > 3.5)
            Predict: 1787.0
        Else (feature 8 > 0.5)
         If (feature 1 <= 2535.5)
          If (feature 0 <= 58.04999923706055)
           If (feature 7 <= 0.5)
            Predict: 2168.945945945946
           Else (feature 7 > 0.5)
            Predict: 2321.8
          Else (feature 0 > 58.04999923706055)
           If (feature 7 <= 0.5)
            Predict: 2017.7096774193549
           Else (feature 7 > 0.5)
            Predict: 2184.564516129032
         Else (feature 1 > 2535.5)
          If (feature 8 <= 1.5)
           If (feature 3 <= 5.825000047683716)
            Predict: 2826.6666666666665
           Else (feature 3 > 5.825000047683716)
            Predict: 2721.0515463917527
          Else (feature 8 > 1.5)
           If (feature 2 <= 0.7450000047683716)
            Predict: 2695.6315789473683
           Else (feature 2 > 0.7450000047683716)
            Predict: 2771.6492146596856
       Else (feature 1 > 2903.5)
        If (feature 1 <= 3416.5)
         If (feature 6 <= 1.5)
          If (feature 2 <= 0.7450000047683716)
           If (feature 8 <= 4.5)
            Predict: 3025.0
           Else (feature 8 > 4.5)
            Predict: 2958.6666666666665
          Else (feature 2 > 0.7450000047683716)
           If (feature 5 <= 3.9450000524520874)
            Predict: 3218.4285714285716
           Else (feature 5 > 3.9450000524520874)
            Predict: 3134.608695652174
         Else (feature 6 > 1.5)
          If (feature 7 <= 2.5)
           If (feature 0 <= 60.04999923706055)
            Predict: 3056.435483870968
           Else (feature 0 > 60.04999923706055)
            Predict: 3223.6666666666665
          Else (feature 7 > 2.5)
           If (feature 3 <= 6.065000057220459)
            Predict: 3145.8582089552237
           Else (feature 3 > 6.065000057220459)
            Predict: 3077.1666666666665
        Else (feature 1 > 3416.5)
         If (feature 2 <= 0.875)
          If (feature 5 <= 3.7350000143051147)
           If (feature 7 <= 2.5)
            Predict: 3513.705882352941
           Else (feature 7 > 2.5)
            Predict: 3642.183673469388
          Else (feature 5 > 3.7350000143051147)
           If (feature 3 <= 6.065000057220459)
            Predict: 3567.3846153846152
           Else (feature 3 > 6.065000057220459)
            Predict: 3556.0
         Else (feature 2 > 0.875)
          If (feature 0 <= 57.04999923706055)
           If (feature 5 <= 3.9450000524520874)
            Predict: 3579.0
           Else (feature 5 > 3.9450000524520874)
            Predict: 3760.2
          Else (feature 0 > 57.04999923706055)
           If (feature 7 <= 4.5)
            Predict: 3749.5416666666665
           Else (feature 7 > 4.5)
            Predict: 3638.6666666666665
     Else (feature 4 > 6.055000066757202)
      If (feature 1 <= 3416.5)
       If (feature 5 <= 3.7350000143051147)
        If (feature 2 <= 0.875)
         If (feature 1 <= 2535.5)
          If (feature 4 <= 6.234999895095825)
           If (feature 5 <= 3.5449999570846558)
            Predict: 2362.75
           Else (feature 5 > 3.5449999570846558)
            Predict: 2235.2380952380954
          Else (feature 4 > 6.234999895095825)
           Predict: 2033.0
         Else (feature 1 > 2535.5)
          If (feature 1 <= 2903.5)
           If (feature 7 <= 2.5)
            Predict: 2693.85
           Else (feature 7 > 2.5)
            Predict: 2801.4347826086955
          Else (feature 1 > 2903.5)
           If (feature 5 <= 3.4550000429153442)
            Predict: 2940.0
           Else (feature 5 > 3.4550000429153442)
            Predict: 3128.7241379310344
        Else (feature 2 > 0.875)
         If (feature 1 <= 2184.5)
          If (feature 0 <= 60.04999923706055)
           Predict: 1547.0
          Else (feature 0 > 60.04999923706055)
           Predict: 1939.0
         Else (feature 1 > 2184.5)
          If (feature 0 <= 55.04999923706055)
           Predict: 2700.0
          Else (feature 0 > 55.04999923706055)
           If (feature 0 <= 60.04999923706055)
            Predict: 3330.4666666666667
           Else (feature 0 > 60.04999923706055)
            Predict: 3151.9
       Else (feature 5 > 3.7350000143051147)
        If (feature 8 <= 0.5)
         If (feature 6 <= 0.5)
          If (feature 5 <= 3.8649998903274536)
           If (feature 1 <= 1649.5)
            Predict: 1637.0
           Else (feature 1 > 1649.5)
            Predict: 1705.0
          Else (feature 5 > 3.8649998903274536)
           If (feature 1 <= 2535.5)
            Predict: 2052.75
           Else (feature 1 > 2535.5)
            Predict: 2808.25
         Else (feature 6 > 0.5)
          If (feature 1 <= 2535.5)
           If (feature 7 <= 2.5)
            Predict: 2050.3333333333335
           Else (feature 7 > 2.5)
            Predict: 2277.8
          Else (feature 1 > 2535.5)
           If (feature 3 <= 6.375)
            Predict: 2802.6363636363635
           Else (feature 3 > 6.375)
            Predict: 3090.25
        Else (feature 8 > 0.5)
         If (feature 4 <= 6.234999895095825)
          If (feature 7 <= 2.5)
           If (feature 1 <= 2903.5)
            Predict: 2722.728813559322
           Else (feature 1 > 2903.5)
            Predict: 3195.576271186441
          Else (feature 7 > 2.5)
           If (feature 0 <= 61.95000076293945)
            Predict: 3222.204081632653
           Else (feature 0 > 61.95000076293945)
            Predict: 3087.5882352941176
         Else (feature 4 > 6.234999895095825)
          If (feature 6 <= 0.5)
           If (feature 7 <= 1.5)
            Predict: 3074.3333333333335
           Else (feature 7 > 1.5)
            Predict: 2966.6666666666665
          Else (feature 6 > 0.5)
           If (feature 1 <= 2903.5)
            Predict: 2844.5
           Else (feature 1 > 2903.5)
            Predict: 3229.390804597701
      Else (feature 1 > 3416.5)
       If (feature 8 <= 3.5)
        If (feature 4 <= 6.234999895095825)
         If (feature 5 <= 3.8649998903274536)
          If (feature 7 <= 2.5)
           If (feature 8 <= 2.5)
            Predict: 3563.451612903226
           Else (feature 8 > 2.5)
            Predict: 3692.2580645161293
          Else (feature 7 > 2.5)
           If (feature 6 <= 3.5)
            Predict: 3717.9850746268658
           Else (feature 6 > 3.5)
            Predict: 3768.4
         Else (feature 5 > 3.8649998903274536)
          If (feature 2 <= 1.0049999952316284)
           If (feature 8 <= 1.5)
            Predict: 3606.0975609756097
           Else (feature 8 > 1.5)
            Predict: 3665.6703296703295
          Else (feature 2 > 1.0049999952316284)
           If (feature 5 <= 3.9950000047683716)
            Predict: 3612.25
           Else (feature 5 > 3.9950000047683716)
            Predict: 3775.3076923076924
        Else (feature 4 > 6.234999895095825)
         If (feature 6 <= 1.5)
          If (feature 3 <= 6.244999885559082)
           If (feature 7 <= 3.5)
            Predict: 3802.3333333333335
           Else (feature 7 > 3.5)
            Predict: 3626.0
          Else (feature 3 > 6.244999885559082)
           If (feature 7 <= 2.5)
            Predict: 3735.4
           Else (feature 7 > 2.5)
            Predict: 3832.5666666666666
         Else (feature 6 > 1.5)
          If (feature 2 <= 1.0049999952316284)
           If (feature 7 <= 1.5)
            Predict: 3637.1176470588234
           Else (feature 7 > 1.5)
            Predict: 3724.2448979591836
          Else (feature 2 > 1.0049999952316284)
           If (feature 0 <= 61.95000076293945)
            Predict: 3692.6666666666665
           Else (feature 0 > 61.95000076293945)
            Predict: 3565.8
       Else (feature 8 > 3.5)
        If (feature 8 <= 4.5)
         If (feature 0 <= 56.04999923706055)
          If (feature 3 <= 6.244999885559082)
           Predict: 3651.6666666666665
          Else (feature 3 > 6.244999885559082)
           If (feature 5 <= 3.8649998903274536)
            Predict: 3921.0
           Else (feature 5 > 3.8649998903274536)
            Predict: 3669.0
         Else (feature 0 > 56.04999923706055)
          If (feature 5 <= 3.59499990940094)
           Predict: 3798.0
          Else (feature 5 > 3.59499990940094)
           If (feature 3 <= 6.244999885559082)
            Predict: 3602.0476190476193
           Else (feature 3 > 6.244999885559082)
            Predict: 3671.5
        Else (feature 8 > 4.5)
         If (feature 6 <= 3.5)
          If (feature 5 <= 3.8649998903274536)
           If (feature 7 <= 0.5)
            Predict: 3746.0
           Else (feature 7 > 0.5)
            Predict: 3865.1666666666665
          Else (feature 5 > 3.8649998903274536)
           Predict: 3718.0
         Else (feature 6 > 3.5)
          If (feature 2 <= 0.875)
           Predict: 3570.0
          Else (feature 2 > 0.875)
           Predict: 3558.0
    Else (feature 1 > 3939.0)
     If (feature 1 <= 5457.5)
      If (feature 5 <= 3.9450000524520874)
       If (feature 8 <= 3.5)
        If (feature 5 <= 3.7350000143051147)
         If (feature 1 <= 4386.5)
          If (feature 0 <= 56.04999923706055)
           If (feature 7 <= 4.5)
            Predict: 3972.4444444444443
           Else (feature 7 > 4.5)
            Predict: 4058.3703703703704
          Else (feature 0 > 56.04999923706055)
           If (feature 3 <= 6.375)
            Predict: 4171.476190476191
           Else (feature 3 > 6.375)
            Predict: 4385.0
         Else (feature 1 > 4386.5)
          If (feature 3 <= 6.244999885559082)
           If (feature 8 <= 2.5)
            Predict: 4693.0
           Else (feature 8 > 2.5)
            Predict: 4703.2
          Else (feature 3 > 6.244999885559082)
           If (feature 0 <= 61.95000076293945)
            Predict: 4564.5
           Else (feature 0 > 61.95000076293945)
            Predict: 4528.25
        Else (feature 5 > 3.7350000143051147)
         If (feature 7 <= 3.5)
          If (feature 3 <= 6.375)
           If (feature 2 <= 0.9150000214576721)
            Predict: 4236.328
           Else (feature 2 > 0.9150000214576721)
            Predict: 4346.3432835820895
          Else (feature 3 > 6.375)
           If (feature 8 <= 1.5)
            Predict: 4120.5
           Else (feature 8 > 1.5)
            Predict: 4774.0
         Else (feature 7 > 3.5)
          If (feature 6 <= 0.5)
           If (feature 1 <= 4386.5)
            Predict: 4098.5
           Else (feature 1 > 4386.5)
            Predict: 4828.083333333333
          Else (feature 6 > 0.5)
           If (feature 2 <= 1.0049999952316284)
            Predict: 4407.340182648401
           Else (feature 2 > 1.0049999952316284)
            Predict: 4809.454545454545
       Else (feature 8 > 3.5)
        If (feature 4 <= 6.055000066757202)
         If (feature 1 <= 4386.5)
          If (feature 3 <= 5.724999904632568)
           If (feature 0 <= 59.10000038146973)
            Predict: 4276.866666666667
           Else (feature 0 > 59.10000038146973)
            Predict: 4022.0
          Else (feature 3 > 5.724999904632568)
           If (feature 8 <= 4.5)
            Predict: 4074.3684210526317
           Else (feature 8 > 4.5)
            Predict: 4182.891891891892
         Else (feature 1 > 4386.5)
          If (feature 7 <= 4.5)
           If (feature 1 <= 4848.0)
            Predict: 4530.692307692308
           Else (feature 1 > 4848.0)
            Predict: 4930.0
          Else (feature 7 > 4.5)
           If (feature 1 <= 4848.0)
            Predict: 4634.0
           Else (feature 1 > 4848.0)
            Predict: 5067.0
        Else (feature 4 > 6.055000066757202)
         If (feature 4 <= 6.234999895095825)
          If (feature 0 <= 53.95000076293945)
           Predict: 5029.0
          Else (feature 0 > 53.95000076293945)
           If (feature 2 <= 0.875)
            Predict: 4478.444444444444
           Else (feature 2 > 0.875)
            Predict: 4666.7109375
         Else (feature 4 > 6.234999895095825)
          If (feature 7 <= 1.5)
           If (feature 1 <= 4386.5)
            Predict: 4095.5
           Else (feature 1 > 4386.5)
            Predict: 4740.5
          Else (feature 7 > 1.5)
           If (feature 7 <= 4.5)
            Predict: 4902.217391304348
           Else (feature 7 > 4.5)
            Predict: 5176.666666666667
      Else (feature 5 > 3.9450000524520874)
       If (feature 3 <= 6.4649999141693115)
        If (feature 6 <= 3.5)
         If (feature 0 <= 56.04999923706055)
          If (feature 6 <= 1.5)
           If (feature 1 <= 4848.0)
            Predict: 4254.4
           Else (feature 1 > 4848.0)
            Predict: 5076.333333333333
          Else (feature 6 > 1.5)
           If (feature 1 <= 4848.0)
            Predict: 4505.533333333334
           Else (feature 1 > 4848.0)
            Predict: 5019.25
         Else (feature 0 > 56.04999923706055)
          If (feature 8 <= 1.5)
           If (feature 0 <= 57.04999923706055)
            Predict: 4471.0
           Else (feature 0 > 57.04999923706055)
            Predict: 4275.853932584269
          Else (feature 8 > 1.5)
           If (feature 3 <= 6.065000057220459)
            Predict: 4200.5
           Else (feature 3 > 6.065000057220459)
            Predict: 4761.83984375
        Else (feature 6 > 3.5)
         If (feature 8 <= 1.5)
          If (feature 0 <= 56.04999923706055)
           Predict: 4249.0
          Else (feature 0 > 56.04999923706055)
           If (feature 2 <= 1.0049999952316284)
            Predict: 4119.0
           Else (feature 2 > 1.0049999952316284)
            Predict: 5206.0
         Else (feature 8 > 1.5)
          If (feature 1 <= 4848.0)
           If (feature 4 <= 6.234999895095825)
            Predict: 4378.0
           Else (feature 4 > 6.234999895095825)
            Predict: 4544.8
          Else (feature 1 > 4848.0)
           If (feature 8 <= 2.5)
            Predict: 5168.666666666667
           Else (feature 8 > 2.5)
            Predict: 5101.5
       Else (feature 3 > 6.4649999141693115)
        If (feature 1 <= 4386.5)
         Predict: 4381.0
        Else (feature 1 > 4386.5)
         If (feature 5 <= 3.9950000047683716)
          If (feature 0 <= 58.04999923706055)
           Predict: 5169.0
          Else (feature 0 > 58.04999923706055)
           Predict: 5038.0
         Else (feature 5 > 3.9950000047683716)
          Predict: 5364.0
     Else (feature 1 > 5457.5)
      If (feature 1 <= 7304.0)
       If (feature 1 <= 6171.0)
        If (feature 0 <= 56.04999923706055)
         If (feature 3 <= 6.4649999141693115)
          If (feature 8 <= 2.5)
           If (feature 0 <= 55.04999923706055)
            Predict: 5490.25
           Else (feature 0 > 55.04999923706055)
            Predict: 5653.428571428572
          Else (feature 8 > 2.5)
           If (feature 3 <= 6.244999885559082)
            Predict: 5678.117647058823
           Else (feature 3 > 6.244999885559082)
            Predict: 5771.238095238095
         Else (feature 3 > 6.4649999141693115)
          Predict: 6108.0
        Else (feature 0 > 56.04999923706055)
         If (feature 2 <= 1.0049999952316284)
          If (feature 8 <= 2.5)
           If (feature 5 <= 3.9450000524520874)
            Predict: 5673.714285714285
           Else (feature 5 > 3.9450000524520874)
            Predict: 5535.6
          Else (feature 8 > 2.5)
           If (feature 5 <= 3.8649998903274536)
            Predict: 5705.913043478261
           Else (feature 5 > 3.8649998903274536)
            Predict: 5838.945945945946
         Else (feature 2 > 1.0049999952316284)
          If (feature 2 <= 1.0149999856948853)
           If (feature 5 <= 4.125)
            Predict: 5903.229508196721
           Else (feature 5 > 4.125)
            Predict: 6089.0
          Else (feature 2 > 1.0149999856948853)
           If (feature 7 <= 4.5)
            Predict: 5730.9
           Else (feature 7 > 4.5)
            Predict: 5551.833333333333
       Else (feature 1 > 6171.0)
        If (feature 8 <= 4.5)
         If (feature 3 <= 6.375)
          If (feature 2 <= 0.9150000214576721)
           Predict: 6741.0
          Else (feature 2 > 0.9150000214576721)
           If (feature 6 <= 1.5)
            Predict: 6375.170731707317
           Else (feature 6 > 1.5)
            Predict: 6519.153846153846
         Else (feature 3 > 6.375)
          If (feature 6 <= 2.5)
           If (feature 8 <= 3.5)
            Predict: 6681.571428571428
           Else (feature 8 > 3.5)
            Predict: 6987.0
          Else (feature 6 > 2.5)
           If (feature 0 <= 55.04999923706055)
            Predict: 7056.0
           Else (feature 0 > 55.04999923706055)
            Predict: 6630.576923076923
        Else (feature 8 > 4.5)
         If (feature 7 <= 3.5)
          If (feature 5 <= 3.9950000047683716)
           If (feature 5 <= 3.9450000524520874)
            Predict: 6371.0
           Else (feature 5 > 3.9450000524520874)
            Predict: 6209.0
          Else (feature 5 > 3.9950000047683716)
           If (feature 5 <= 4.045000076293945)
            Predict: 6968.0
           Else (feature 5 > 4.045000076293945)
            Predict: 6448.5
         Else (feature 7 > 3.5)
          If (feature 3 <= 6.065000057220459)
           Predict: 6338.0
          Else (feature 3 > 6.065000057220459)
           If (feature 7 <= 4.5)
            Predict: 7053.2
           Else (feature 7 > 4.5)
            Predict: 6867.333333333333
      Else (feature 1 > 7304.0)
       If (feature 7 <= 5.5)
        If (feature 5 <= 4.125)
         If (feature 8 <= 4.5)
          If (feature 2 <= 1.0049999952316284)
           If (feature 0 <= 57.04999923706055)
            Predict: 7582.0
           Else (feature 0 > 57.04999923706055)
            Predict: 7374.0
          Else (feature 2 > 1.0049999952316284)
           If (feature 7 <= 4.5)
            Predict: 7937.5
           Else (feature 7 > 4.5)
            Predict: 7632.285714285715
         Else (feature 8 > 4.5)
          If (feature 1 <= 8817.0)
           If (feature 5 <= 4.045000076293945)
            Predict: 7948.041666666667
           Else (feature 5 > 4.045000076293945)
            Predict: 8554.2
          Else (feature 1 > 8817.0)
           If (feature 5 <= 3.9950000047683716)
            Predict: 9706.57142857143
           Else (feature 5 > 3.9950000047683716)
            Predict: 9184.5
        Else (feature 5 > 4.125)
         Predict: 14918.0
       Else (feature 7 > 5.5)
        If (feature 8 <= 5.5)
         If (feature 1 <= 8817.0)
          If (feature 8 <= 4.5)
           If (feature 0 <= 55.04999923706055)
            Predict: 8333.0
           Else (feature 0 > 55.04999923706055)
            Predict: 7689.583333333333
          Else (feature 8 > 4.5)
           If (feature 5 <= 3.8649998903274536)
            Predict: 8707.0
           Else (feature 5 > 3.8649998903274536)
            Predict: 8282.0
         Else (feature 1 > 8817.0)
          If (feature 2 <= 1.0049999952316284)
           Predict: 9660.0
          Else (feature 2 > 1.0049999952316284)
           If (feature 8 <= 1.5)
            Predict: 10011.0
           Else (feature 8 > 1.5)
            Predict: 9630.2
        Else (feature 8 > 5.5)
         If (feature 2 <= 1.0049999952316284)
          Predict: 16073.0
         Else (feature 2 > 1.0049999952316284)
          If (feature 1 <= 14203.5)
           Predict: 12581.0
          Else (feature 1 > 14203.5)
           Predict: 15575.0
  Else (feature 4 > 6.375)
   If (feature 8 <= 2.5)
    If (feature 4 <= 7.4049999713897705)
     If (feature 1 <= 7304.0)
      If (feature 2 <= 1.125)
       If (feature 5 <= 3.8649998903274536)
        If (feature 8 <= 1.5)
         If (feature 1 <= 3416.5)
          If (feature 1 <= 2535.5)
           If (feature 7 <= 0.5)
            Predict: 1681.0
           Else (feature 7 > 0.5)
            Predict: 2184.0
          Else (feature 1 > 2535.5)
           If (feature 3 <= 6.4649999141693115)
            Predict: 3244.25
           Else (feature 3 > 6.4649999141693115)
            Predict: 3015.5333333333333
         Else (feature 1 > 3416.5)
          If (feature 7 <= 4.5)
           If (feature 5 <= 3.7350000143051147)
            Predict: 3754.6666666666665
           Else (feature 5 > 3.7350000143051147)
            Predict: 3891.315789473684
          Else (feature 7 > 4.5)
           If (feature 0 <= 59.10000038146973)
            Predict: 3862.222222222222
           Else (feature 0 > 59.10000038146973)
            Predict: 4241.727272727273
        Else (feature 8 > 1.5)
         If (feature 6 <= 0.5)
          If (feature 0 <= 61.95000076293945)
           If (feature 7 <= 1.5)
            Predict: 3910.6666666666665
           Else (feature 7 > 1.5)
            Predict: 5018.666666666667
          Else (feature 0 > 61.95000076293945)
           If (feature 1 <= 3416.5)
            Predict: 3051.5
           Else (feature 1 > 3416.5)
            Predict: 4105.714285714285
         Else (feature 6 > 0.5)
          If (feature 7 <= 2.5)
           If (feature 0 <= 55.04999923706055)
            Predict: 3472.0
           Else (feature 0 > 55.04999923706055)
            Predict: 4073.5
          Else (feature 7 > 2.5)
           If (feature 1 <= 4848.0)
            Predict: 4446.727272727273
           Else (feature 1 > 4848.0)
            Predict: 5314.542857142857
       Else (feature 5 > 3.8649998903274536)
        If (feature 8 <= 1.5)
         If (feature 8 <= 0.5)
          If (feature 4 <= 6.684999942779541)
           If (feature 4 <= 6.454999923706055)
            Predict: 2911.6190476190477
           Else (feature 4 > 6.454999923706055)
            Predict: 3280.025641025641
          Else (feature 4 > 6.684999942779541)
           If (feature 6 <= 1.5)
            Predict: 2220.0
           Else (feature 6 > 1.5)
            Predict: 4072.5
         Else (feature 8 > 0.5)
          If (feature 1 <= 4386.5)
           If (feature 1 <= 3939.0)
            Predict: 3656.597989949749
           Else (feature 1 > 3939.0)
            Predict: 4199.136498516321
          Else (feature 1 > 4386.5)
           If (feature 1 <= 4848.0)
            Predict: 4590.450134770889
           Else (feature 1 > 4848.0)
            Predict: 5202.560693641619
        Else (feature 8 > 1.5)
         If (feature 6 <= 3.5)
          If (feature 1 <= 4848.0)
           If (feature 7 <= 1.5)
            Predict: 4117.618556701031
           Else (feature 7 > 1.5)
            Predict: 4577.2303664921465
          Else (feature 1 > 4848.0)
           If (feature 4 <= 6.684999942779541)
            Predict: 5352.757033248082
           Else (feature 4 > 6.684999942779541)
            Predict: 5589.658536585366
         Else (feature 6 > 3.5)
          If (feature 4 <= 6.684999942779541)
           If (feature 2 <= 1.0449999570846558)
            Predict: 5158.571428571428
           Else (feature 2 > 1.0449999570846558)
            Predict: 5375.735294117647
          Else (feature 4 > 6.684999942779541)
           If (feature 7 <= 2.5)
            Predict: 5303.0
           Else (feature 7 > 2.5)
            Predict: 6885.166666666667
      Else (feature 2 > 1.125)
       If (feature 4 <= 6.855000019073486)
        If (feature 8 <= 1.5)
         If (feature 1 <= 4848.0)
          If (feature 8 <= 0.5)
           If (feature 4 <= 6.684999942779541)
            Predict: 3109.5789473684213
           Else (feature 4 > 6.684999942779541)
            Predict: 3811.653846153846
          Else (feature 8 > 0.5)
           If (feature 5 <= 4.125)
            Predict: 4091.809523809524
           Else (feature 5 > 4.125)
            Predict: 4362.5822784810125
         Else (feature 1 > 4848.0)
          If (feature 1 <= 5457.5)
           If (feature 5 <= 4.4049999713897705)
            Predict: 5131.007246376812
           Else (feature 5 > 4.4049999713897705)
            Predict: 5429.0
          Else (feature 1 > 5457.5)
           If (feature 2 <= 1.215000033378601)
            Predict: 5871.736842105263
           Else (feature 2 > 1.215000033378601)
            Predict: 6135.333333333333
        Else (feature 8 > 1.5)
         If (feature 7 <= 1.5)
          If (feature 1 <= 4848.0)
           If (feature 1 <= 4386.5)
            Predict: 4093.3076923076924
           Else (feature 1 > 4386.5)
            Predict: 4687.782608695652
          Else (feature 1 > 4848.0)
           If (feature 1 <= 5457.5)
            Predict: 5189.137931034483
           Else (feature 1 > 5457.5)
            Predict: 5718.0
         Else (feature 7 > 1.5)
          If (feature 1 <= 6171.0)
           If (feature 3 <= 6.555000066757202)
            Predict: 4503.333333333333
           Else (feature 3 > 6.555000066757202)
            Predict: 5495.825806451613
          Else (feature 1 > 6171.0)
           If (feature 0 <= 61.95000076293945)
            Predict: 6732.684782608696
           Else (feature 0 > 61.95000076293945)
            Predict: 6235.2
       Else (feature 4 > 6.855000019073486)
        If (feature 5 <= 4.125)
         If (feature 7 <= 5.5)
          If (feature 8 <= 0.5)
           If (feature 0 <= 59.10000038146973)
            Predict: 3736.5
           Else (feature 0 > 59.10000038146973)
            Predict: 3144.0
          Else (feature 8 > 0.5)
           If (feature 8 <= 1.5)
            Predict: 4940.666666666667
           Else (feature 8 > 1.5)
            Predict: 5617.409090909091
         Else (feature 7 > 5.5)
          If (feature 0 <= 58.04999923706055)
           Predict: 5504.0
          Else (feature 0 > 58.04999923706055)
           If (feature 6 <= 1.5)
            Predict: 3882.5
           Else (feature 6 > 1.5)
            Predict: 4369.333333333333
        Else (feature 5 > 4.125)
         If (feature 3 <= 7.1549999713897705)
          If (feature 7 <= 1.5)
           If (feature 7 <= 0.5)
            Predict: 5037.813953488372
           Else (feature 7 > 0.5)
            Predict: 5591.933333333333
          Else (feature 7 > 1.5)
           If (feature 8 <= 0.5)
            Predict: 4573.925925925926
           Else (feature 8 > 0.5)
            Predict: 6151.110778443114
         Else (feature 3 > 7.1549999713897705)
          If (feature 6 <= 0.5)
           If (feature 1 <= 5457.5)
            Predict: 4860.3
           Else (feature 1 > 5457.5)
            Predict: 6695.125
          Else (feature 6 > 0.5)
           If (feature 1 <= 5457.5)
            Predict: 4499.5
           Else (feature 1 > 5457.5)
            Predict: 6542.942528735633
     Else (feature 1 > 7304.0)
      If (feature 2 <= 1.4049999713897705)
       If (feature 6 <= 3.5)
        If (feature 8 <= 1.5)
         If (feature 4 <= 7.134999990463257)
          If (feature 6 <= 2.5)
           Predict: 7431.0
          Else (feature 6 > 2.5)
           If (feature 3 <= 7.1549999713897705)
            Predict: 7366.571428571428
           Else (feature 3 > 7.1549999713897705)
            Predict: 7357.0
         Else (feature 4 > 7.134999990463257)
          Predict: 8368.666666666666
        Else (feature 8 > 1.5)
         If (feature 4 <= 6.855000019073486)
          If (feature 5 <= 4.045000076293945)
           Predict: 8184.0
          Else (feature 5 > 4.045000076293945)
           If (feature 7 <= 3.5)
            Predict: 7820.5
           Else (feature 7 > 3.5)
            Predict: 7482.818181818182
         Else (feature 4 > 6.855000019073486)
          If (feature 5 <= 4.125)
           Predict: 7338.0
          Else (feature 5 > 4.125)
           If (feature 4 <= 7.134999990463257)
            Predict: 7931.190476190476
           Else (feature 4 > 7.134999990463257)
            Predict: 7879.111111111111
       Else (feature 6 > 3.5)
        If (feature 0 <= 55.04999923706055)
         If (feature 0 <= 53.95000076293945)
          Predict: 9519.0
         Else (feature 0 > 53.95000076293945)
          If (feature 1 <= 8817.0)
           Predict: 7817.666666666667
          Else (feature 1 > 8817.0)
           If (feature 7 <= 4.5)
            Predict: 9331.0
           Else (feature 7 > 4.5)
            Predict: 9548.0
        Else (feature 0 > 55.04999923706055)
         If (feature 2 <= 1.0449999570846558)
          Predict: 7564.0
         Else (feature 2 > 1.0449999570846558)
          If (feature 7 <= 4.5)
           If (feature 1 <= 8817.0)
            Predict: 7732.277777777777
           Else (feature 1 > 8817.0)
            Predict: 8989.0
          Else (feature 7 > 4.5)
           If (feature 4 <= 6.855000019073486)
            Predict: 7508.0
           Else (feature 4 > 6.855000019073486)
            Predict: 8733.6
      Else (feature 2 > 1.4049999713897705)
       If (feature 8 <= 1.5)
        If (feature 3 <= 7.414999961853027)
         If (feature 3 <= 7.1549999713897705)
          If (feature 4 <= 7.134999990463257)
           If (feature 5 <= 4.575000047683716)
            Predict: 8228.25
           Else (feature 5 > 4.575000047683716)
            Predict: 7474.0
          Else (feature 4 > 7.134999990463257)
           Predict: 7495.0
         Else (feature 3 > 7.1549999713897705)
          If (feature 6 <= 1.5)
           If (feature 2 <= 1.524999976158142)
            Predict: 8089.172413793103
           Else (feature 2 > 1.524999976158142)
            Predict: 9045.0
          Else (feature 6 > 1.5)
           If (feature 1 <= 8817.0)
            Predict: 8086.127659574468
           Else (feature 1 > 8817.0)
            Predict: 9552.41935483871
        Else (feature 3 > 7.414999961853027)
         If (feature 0 <= 55.04999923706055)
          If (feature 2 <= 1.524999976158142)
           Predict: 10080.0
          Else (feature 2 > 1.524999976158142)
           Predict: 10937.0
         Else (feature 0 > 55.04999923706055)
          If (feature 7 <= 5.5)
           If (feature 5 <= 4.575000047683716)
            Predict: 8235.0
           Else (feature 5 > 4.575000047683716)
            Predict: 9122.5
          Else (feature 7 > 5.5)
           If (feature 1 <= 10901.0)
            Predict: 9637.0
           Else (feature 1 > 10901.0)
            Predict: 11103.0
       Else (feature 8 > 1.5)
        If (feature 6 <= 2.5)
         If (feature 1 <= 8817.0)
          If (feature 6 <= 0.5)
           If (feature 0 <= 57.04999923706055)
            Predict: 7895.0
           Else (feature 0 > 57.04999923706055)
            Predict: 7631.8
          Else (feature 6 > 0.5)
           If (feature 0 <= 59.10000038146973)
            Predict: 8017.804878048781
           Else (feature 0 > 59.10000038146973)
            Predict: 8546.583333333334
         Else (feature 1 > 8817.0)
          If (feature 5 <= 4.4049999713897705)
           If (feature 7 <= 2.5)
            Predict: 9573.0
           Else (feature 7 > 2.5)
            Predict: 9278.0
          Else (feature 5 > 4.4049999713897705)
           If (feature 1 <= 10901.0)
            Predict: 9958.151515151516
           Else (feature 1 > 10901.0)
            Predict: 11724.113636363636
        Else (feature 6 > 2.5)
         If (feature 1 <= 10901.0)
          If (feature 1 <= 8817.0)
           If (feature 2 <= 1.524999976158142)
            Predict: 8227.65
           Else (feature 2 > 1.524999976158142)
            Predict: 7846.0
          Else (feature 1 > 8817.0)
           If (feature 3 <= 7.414999961853027)
            Predict: 9841.776119402984
           Else (feature 3 > 7.414999961853027)
            Predict: 9547.142857142857
         Else (feature 1 > 10901.0)
          If (feature 1 <= 14203.5)
           If (feature 6 <= 3.5)
            Predict: 11552.894736842105
           Else (feature 6 > 3.5)
            Predict: 12071.69696969697
          Else (feature 1 > 14203.5)
           Predict: 14426.0
    Else (feature 4 > 7.4049999713897705)
     If (feature 2 <= 1.9800000190734863)
      If (feature 7 <= 2.5)
       If (feature 7 <= 0.5)
        If (feature 1 <= 7304.0)
         If (feature 0 <= 57.04999923706055)
          If (feature 0 <= 54.95000076293945)
           Predict: 7155.0
          Else (feature 0 > 54.95000076293945)
           If (feature 0 <= 56.04999923706055)
            Predict: 6031.0
           Else (feature 0 > 56.04999923706055)
            Predict: 6739.2
         Else (feature 0 > 57.04999923706055)
          If (feature 1 <= 6171.0)
           If (feature 6 <= 2.5)
            Predict: 5902.0
           Else (feature 6 > 2.5)
            Predict: 5213.666666666667
          Else (feature 1 > 6171.0)
           Predict: 6996.0
        Else (feature 1 > 7304.0)
         If (feature 8 <= 1.5)
          If (feature 0 <= 58.04999923706055)
           If (feature 0 <= 57.04999923706055)
            Predict: 7662.75
           Else (feature 0 > 57.04999923706055)
            Predict: 7409.0
          Else (feature 0 > 58.04999923706055)
           If (feature 6 <= 1.5)
            Predict: 8109.0
           Else (feature 6 > 1.5)
            Predict: 8257.0
         Else (feature 8 > 1.5)
          If (feature 2 <= 1.524999976158142)
           If (feature 6 <= 3.5)
            Predict: 7623.0
           Else (feature 6 > 3.5)
            Predict: 8049.0
          Else (feature 2 > 1.524999976158142)
           If (feature 0 <= 59.10000038146973)
            Predict: 8662.787878787878
           Else (feature 0 > 59.10000038146973)
            Predict: 8307.363636363636
       Else (feature 7 > 0.5)
        If (feature 1 <= 8817.0)
         If (feature 1 <= 7304.0)
          If (feature 4 <= 7.9100000858306885)
           If (feature 1 <= 6171.0)
            Predict: 5713.333333333333
           Else (feature 1 > 6171.0)
            Predict: 6901.931034482759
          Else (feature 4 > 7.9100000858306885)
           Predict: 5045.0
         Else (feature 1 > 7304.0)
          If (feature 0 <= 59.10000038146973)
           If (feature 6 <= 1.5)
            Predict: 7601.625
           Else (feature 6 > 1.5)
            Predict: 8038.477272727273
          Else (feature 0 > 59.10000038146973)
           If (feature 7 <= 1.5)
            Predict: 8184.133333333333
           Else (feature 7 > 1.5)
            Predict: 7771.5
        Else (feature 1 > 8817.0)
         If (feature 7 <= 1.5)
          If (feature 4 <= 7.9100000858306885)
           If (feature 0 <= 59.10000038146973)
            Predict: 9938.070175438597
           Else (feature 0 > 59.10000038146973)
            Predict: 10493.57142857143
          Else (feature 4 > 7.9100000858306885)
           If (feature 0 <= 53.95000076293945)
            Predict: 13367.0
           Else (feature 0 > 53.95000076293945)
            Predict: 14581.0
         Else (feature 7 > 1.5)
          If (feature 4 <= 7.9100000858306885)
           If (feature 5 <= 4.575000047683716)
            Predict: 9913.967741935483
           Else (feature 5 > 4.575000047683716)
            Predict: 11014.64705882353
          Else (feature 4 > 7.9100000858306885)
           If (feature 8 <= 1.5)
            Predict: 13099.0
           Else (feature 8 > 1.5)
            Predict: 14273.4
      Else (feature 7 > 2.5)
       If (feature 5 <= 4.575000047683716)
        If (feature 0 <= 59.10000038146973)
         If (feature 2 <= 1.524999976158142)
          If (feature 1 <= 10901.0)
           If (feature 1 <= 8817.0)
            Predict: 8217.2
           Else (feature 1 > 8817.0)
            Predict: 9556.5625
          Else (feature 1 > 10901.0)
           If (feature 8 <= 1.5)
            Predict: 10935.0
           Else (feature 8 > 1.5)
            Predict: 12320.884615384615
         Else (feature 2 > 1.524999976158142)
          If (feature 1 <= 10901.0)
           If (feature 1 <= 8817.0)
            Predict: 8627.0
           Else (feature 1 > 8817.0)
            Predict: 10438.764705882353
          Else (feature 1 > 10901.0)
           If (feature 6 <= 1.5)
            Predict: 13187.0
           Else (feature 6 > 1.5)
            Predict: 12078.88
        Else (feature 0 > 59.10000038146973)
         If (feature 1 <= 8817.0)
          If (feature 1 <= 7304.0)
           If (feature 0 <= 60.04999923706055)
            Predict: 6990.0
           Else (feature 0 > 60.04999923706055)
            Predict: 5992.5
          Else (feature 1 > 7304.0)
           If (feature 7 <= 5.5)
            Predict: 8216.666666666666
           Else (feature 7 > 5.5)
            Predict: 8322.666666666666
         Else (feature 1 > 8817.0)
          If (feature 1 <= 10901.0)
           If (feature 3 <= 7.414999961853027)
            Predict: 9181.5
           Else (feature 3 > 7.414999961853027)
            Predict: 10237.0
          Else (feature 1 > 10901.0)
           If (feature 7 <= 4.5)
            Predict: 11204.0
           Else (feature 7 > 4.5)
            Predict: 12331.882352941177
       Else (feature 5 > 4.575000047683716)
        If (feature 1 <= 10901.0)
         If (feature 6 <= 1.5)
          If (feature 1 <= 7304.0)
           If (feature 1 <= 6171.0)
            Predict: 5793.111111111111
           Else (feature 1 > 6171.0)
            Predict: 6971.0
          Else (feature 1 > 7304.0)
           If (feature 7 <= 3.5)
            Predict: 9345.0
           Else (feature 7 > 3.5)
            Predict: 8220.0
         Else (feature 6 > 1.5)
          If (feature 3 <= 7.414999961853027)
           If (feature 6 <= 2.5)
            Predict: 10180.0
           Else (feature 6 > 2.5)
            Predict: 9658.333333333334
          Else (feature 3 > 7.414999961853027)
           If (feature 0 <= 56.04999923706055)
            Predict: 9866.608695652174
           Else (feature 0 > 56.04999923706055)
            Predict: 8918.74
        Else (feature 1 > 10901.0)
         If (feature 1 <= 14203.5)
          If (feature 0 <= 59.10000038146973)
           If (feature 0 <= 56.04999923706055)
            Predict: 12309.142857142857
           Else (feature 0 > 56.04999923706055)
            Predict: 12100.754716981131
          Else (feature 0 > 59.10000038146973)
           If (feature 0 <= 61.95000076293945)
            Predict: 13265.363636363636
           Else (feature 0 > 61.95000076293945)
            Predict: 12392.75
         Else (feature 1 > 14203.5)
          If (feature 5 <= 4.884999990463257)
           If (feature 3 <= 7.924999952316284)
            Predict: 15552.115384615385
           Else (feature 3 > 7.924999952316284)
            Predict: 18306.0
          Else (feature 5 > 4.884999990463257)
           If (feature 4 <= 7.9100000858306885)
            Predict: 16232.0
           Else (feature 4 > 7.9100000858306885)
            Predict: 17509.0
     Else (feature 2 > 1.9800000190734863)
      If (feature 8 <= 0.5)
       If (feature 3 <= 7.924999952316284)
        If (feature 7 <= 1.5)
         If (feature 0 <= 57.04999923706055)
          Predict: 5405.0
         Else (feature 0 > 57.04999923706055)
          Predict: 5676.666666666667
        Else (feature 7 > 1.5)
         If (feature 6 <= 0.5)
          If (feature 1 <= 6171.0)
           If (feature 0 <= 54.95000076293945)
            Predict: 5914.0
           Else (feature 0 > 54.95000076293945)
            Predict: 6002.0
          Else (feature 1 > 6171.0)
           If (feature 7 <= 3.5)
            Predict: 6353.75
           Else (feature 7 > 3.5)
            Predict: 6775.5
         Else (feature 6 > 0.5)
          Predict: 7959.0
       Else (feature 3 > 7.924999952316284)
        If (feature 1 <= 14203.5)
         If (feature 7 <= 0.5)
          If (feature 6 <= 2.5)
           If (feature 0 <= 58.04999923706055)
            Predict: 6458.642857142857
           Else (feature 0 > 58.04999923706055)
            Predict: 10437.0
          Else (feature 6 > 2.5)
           Predict: 5089.0
         Else (feature 7 > 0.5)
          If (feature 6 <= 0.5)
           If (feature 1 <= 7304.0)
            Predict: 6901.933333333333
           Else (feature 1 > 7304.0)
            Predict: 8155.4
          Else (feature 6 > 0.5)
           If (feature 6 <= 1.5)
            Predict: 11548.0
           Else (feature 6 > 1.5)
            Predict: 9225.34375
        Else (feature 1 > 14203.5)
         Predict: 17210.916666666668
      Else (feature 8 > 0.5)
       If (feature 6 <= 0.5)
        If (feature 0 <= 56.04999923706055)
         If (feature 3 <= 7.924999952316284)
          If (feature 7 <= 1.5)
           If (feature 1 <= 8817.0)
            Predict: 8561.25
           Else (feature 1 > 8817.0)
            Predict: 9802.0
          Else (feature 7 > 1.5)
           If (feature 1 <= 10901.0)
            Predict: 10184.0
           Else (feature 1 > 10901.0)
            Predict: 11677.75
         Else (feature 3 > 7.924999952316284)
          If (feature 7 <= 3.5)
           If (feature 1 <= 10901.0)
            Predict: 10465.666666666666
           Else (feature 1 > 10901.0)
            Predict: 13083.6
          Else (feature 7 > 3.5)
           Predict: 16386.0
        Else (feature 0 > 56.04999923706055)
         If (feature 7 <= 3.5)
          If (feature 1 <= 14203.5)
           If (feature 1 <= 10901.0)
            Predict: 10684.333333333334
           Else (feature 1 > 10901.0)
            Predict: 12925.166666666666
          Else (feature 1 > 14203.5)
           If (feature 3 <= 7.924999952316284)
            Predict: 14760.75
           Else (feature 3 > 7.924999952316284)
            Predict: 15660.5
         Else (feature 7 > 3.5)
          If (feature 0 <= 58.04999923706055)
           If (feature 4 <= 7.9100000858306885)
            Predict: 16150.666666666666
           Else (feature 4 > 7.9100000858306885)
            Predict: 17405.0
          Else (feature 0 > 58.04999923706055)
           If (feature 7 <= 4.5)
            Predict: 13420.0
           Else (feature 7 > 4.5)
            Predict: 14492.181818181818
       Else (feature 6 > 0.5)
        If (feature 6 <= 3.5)
         If (feature 7 <= 0.5)
          If (feature 3 <= 7.924999952316284)
           Predict: 11125.666666666666
          Else (feature 3 > 7.924999952316284)
           If (feature 6 <= 2.5)
            Predict: 12678.04347826087
           Else (feature 6 > 2.5)
            Predict: 13334.028169014084
         Else (feature 7 > 0.5)
          If (feature 1 <= 14203.5)
           If (feature 1 <= 10901.0)
            Predict: 10140.0
           Else (feature 1 > 10901.0)
            Predict: 12703.81690140845
          Else (feature 1 > 14203.5)
           If (feature 8 <= 1.5)
            Predict: 16331.265780730897
           Else (feature 8 > 1.5)
            Predict: 16866.814814814814
        Else (feature 6 > 3.5)
         If (feature 7 <= 1.5)
          If (feature 7 <= 0.5)
           If (feature 0 <= 54.95000076293945)
            Predict: 15113.666666666666
           Else (feature 0 > 54.95000076293945)
            Predict: 13424.871794871795
          Else (feature 7 > 0.5)
           If (feature 4 <= 7.9100000858306885)
            Predict: 16287.0
           Else (feature 4 > 7.9100000858306885)
            Predict: 14934.234567901234
         Else (feature 7 > 1.5)
          If (feature 1 <= 14203.5)
           If (feature 8 <= 1.5)
            Predict: 12715.9375
           Else (feature 8 > 1.5)
            Predict: 12892.2
          Else (feature 1 > 14203.5)
           If (feature 8 <= 1.5)
            Predict: 17022.353658536584
           Else (feature 8 > 1.5)
            Predict: 17384.28
   Else (feature 8 > 2.5)
    If (feature 1 <= 10901.0)
     If (feature 2 <= 1.4049999713897705)
      If (feature 1 <= 7304.0)
       If (feature 1 <= 6171.0)
        If (feature 1 <= 4848.0)
         If (feature 3 <= 5.204999923706055)
          Predict: 2075.0
         Else (feature 3 > 5.204999923706055)
          If (feature 5 <= 3.9450000524520874)
           If (feature 3 <= 6.4649999141693115)
            Predict: 4355.6875
           Else (feature 3 > 6.4649999141693115)
            Predict: 3901.810810810811
          Else (feature 5 > 3.9450000524520874)
           If (feature 7 <= 0.5)
            Predict: 4274.428571428572
           Else (feature 7 > 0.5)
            Predict: 4541.854166666667
        Else (feature 1 > 4848.0)
         If (feature 1 <= 5457.5)
          If (feature 7 <= 0.5)
           If (feature 3 <= 6.855000019073486)
            Predict: 5048.8125
           Else (feature 3 > 6.855000019073486)
            Predict: 5162.777777777777
          Else (feature 7 > 0.5)
           If (feature 4 <= 6.855000019073486)
            Predict: 5182.851851851852
           Else (feature 4 > 6.855000019073486)
            Predict: 5353.3
         Else (feature 1 > 5457.5)
          If (feature 3 <= 6.4649999141693115)
           If (feature 6 <= 1.5)
            Predict: 5766.444444444444
           Else (feature 6 > 1.5)
            Predict: 5917.361904761905
          Else (feature 3 > 6.4649999141693115)
           If (feature 8 <= 3.5)
            Predict: 5847.6769911504425
           Else (feature 8 > 3.5)
            Predict: 5793.214285714285
       Else (feature 1 > 6171.0)
        If (feature 0 <= 57.95000076293945)
         If (feature 3 <= 7.1549999713897705)
          If (feature 7 <= 1.5)
           If (feature 8 <= 4.5)
            Predict: 6594.451612903225
           Else (feature 8 > 4.5)
            Predict: 6838.4
          Else (feature 7 > 1.5)
           If (feature 2 <= 1.125)
            Predict: 6760.7405063291135
           Else (feature 2 > 1.125)
            Predict: 6972.7464788732395
         Else (feature 3 > 7.1549999713897705)
          If (feature 8 <= 3.5)
           If (feature 4 <= 7.134999990463257)
            Predict: 6506.666666666667
           Else (feature 4 > 7.134999990463257)
            Predict: 6366.0
          Else (feature 8 > 3.5)
           Predict: 6973.0
        Else (feature 0 > 57.95000076293945)
         If (feature 7 <= 5.5)
          If (feature 5 <= 4.045000076293945)
           If (feature 5 <= 3.9950000047683716)
            Predict: 6679.222222222223
           Else (feature 5 > 3.9950000047683716)
            Predict: 6576.714285714285
          Else (feature 5 > 4.045000076293945)
           If (feature 8 <= 4.5)
            Predict: 6798.729281767955
           Else (feature 8 > 4.5)
            Predict: 6483.4
         Else (feature 7 > 5.5)
          If (feature 8 <= 3.5)
           If (feature 5 <= 3.9450000524520874)
            Predict: 6961.166666666667
           Else (feature 5 > 3.9450000524520874)
            Predict: 6682.6
          Else (feature 8 > 3.5)
           If (feature 6 <= 1.5)
            Predict: 7229.666666666667
           Else (feature 6 > 1.5)
            Predict: 7041.0
      Else (feature 1 > 7304.0)
       If (feature 2 <= 1.125)
        If (feature 1 <= 8817.0)
         If (feature 8 <= 4.5)
          If (feature 3 <= 6.555000066757202)
           If (feature 2 <= 1.0449999570846558)
            Predict: 7711.703947368421
           Else (feature 2 > 1.0449999570846558)
            Predict: 7846.2
          Else (feature 3 > 6.555000066757202)
           If (feature 8 <= 3.5)
            Predict: 7774.722222222223
           Else (feature 8 > 3.5)
            Predict: 7940.307692307692
         Else (feature 8 > 4.5)
          If (feature 6 <= 1.5)
           If (feature 5 <= 4.045000076293945)
            Predict: 7440.0
           Else (feature 5 > 4.045000076293945)
            Predict: 7800.0
          Else (feature 6 > 1.5)
           If (feature 0 <= 59.10000038146973)
            Predict: 8305.15671641791
           Else (feature 0 > 59.10000038146973)
            Predict: 7989.652173913043
        Else (feature 1 > 8817.0)
         If (feature 7 <= 3.5)
          If (feature 5 <= 3.9950000047683716)
           If (feature 0 <= 56.04999923706055)
            Predict: 9063.0
           Else (feature 0 > 56.04999923706055)
            Predict: 8883.333333333334
          Else (feature 5 > 3.9950000047683716)
           If (feature 8 <= 6.5)
            Predict: 9174.260869565218
           Else (feature 8 > 6.5)
            Predict: 9504.0
         Else (feature 7 > 3.5)
          If (feature 3 <= 6.4649999141693115)
           If (feature 4 <= 6.454999923706055)
            Predict: 9775.82142857143
           Else (feature 4 > 6.454999923706055)
            Predict: 10335.652173913044
          Else (feature 3 > 6.4649999141693115)
           If (feature 0 <= 54.95000076293945)
            Predict: 9091.0
           Else (feature 0 > 54.95000076293945)
            Predict: 9784.327586206897
       Else (feature 2 > 1.125)
        If (feature 1 <= 8817.0)
         If (feature 8 <= 3.5)
          If (feature 7 <= 3.5)
           If (feature 7 <= 2.5)
            Predict: 7840.462962962963
           Else (feature 7 > 2.5)
            Predict: 7976.103448275862
          Else (feature 7 > 3.5)
           If (feature 3 <= 6.855000019073486)
            Predict: 8165.108695652174
           Else (feature 3 > 6.855000019073486)
            Predict: 8420.363636363636
         Else (feature 8 > 3.5)
          If (feature 5 <= 4.045000076293945)
           Predict: 8032.625
          Else (feature 5 > 4.045000076293945)
           If (feature 0 <= 54.95000076293945)
            Predict: 7954.545454545455
           Else (feature 0 > 54.95000076293945)
            Predict: 8275.917241379311
        Else (feature 1 > 8817.0)
         If (feature 5 <= 4.234999895095825)
          If (feature 7 <= 3.5)
           If (feature 2 <= 1.215000033378601)
            Predict: 9398.833333333334
           Else (feature 2 > 1.215000033378601)
            Predict: 9819.592592592593
          Else (feature 7 > 3.5)
           If (feature 0 <= 54.95000076293945)
            Predict: 10742.0
           Else (feature 0 > 54.95000076293945)
            Predict: 9706.977777777778
         Else (feature 5 > 4.234999895095825)
          If (feature 7 <= 5.5)
           If (feature 3 <= 7.1549999713897705)
            Predict: 9791.186170212766
           Else (feature 3 > 7.1549999713897705)
            Predict: 10183.333333333334
          Else (feature 7 > 5.5)
           Predict: 10474.833333333334
     Else (feature 2 > 1.4049999713897705)
      If (feature 5 <= 4.575000047683716)
       If (feature 7 <= 0.5)
        If (feature 1 <= 8817.0)
         If (feature 1 <= 7304.0)
          If (feature 6 <= 2.5)
           If (feature 3 <= 7.1549999713897705)
            Predict: 6819.0
           Else (feature 3 > 7.1549999713897705)
            Predict: 7128.25
          Else (feature 6 > 2.5)
           If (feature 3 <= 7.414999961853027)
            Predict: 6479.333333333333
           Else (feature 3 > 7.414999961853027)
            Predict: 6194.0
         Else (feature 1 > 7304.0)
          If (feature 8 <= 3.5)
           If (feature 0 <= 59.10000038146973)
            Predict: 7608.4
           Else (feature 0 > 59.10000038146973)
            Predict: 7439.333333333333
          Else (feature 8 > 3.5)
           If (feature 0 <= 55.04999923706055)
            Predict: 7773.0
           Else (feature 0 > 55.04999923706055)
            Predict: 8319.371428571429
        Else (feature 1 > 8817.0)
         If (feature 8 <= 5.5)
          If (feature 2 <= 1.524999976158142)
           If (feature 6 <= 3.5)
            Predict: 8913.0
           Else (feature 6 > 3.5)
            Predict: 9334.0
          Else (feature 2 > 1.524999976158142)
           If (feature 0 <= 60.04999923706055)
            Predict: 8959.0
           Else (feature 0 > 60.04999923706055)
            Predict: 9314.0
         Else (feature 8 > 5.5)
          If (feature 2 <= 1.524999976158142)
           Predict: 9814.0
          Else (feature 2 > 1.524999976158142)
           Predict: 9435.333333333334
       Else (feature 7 > 0.5)
        If (feature 1 <= 8817.0)
         If (feature 1 <= 7304.0)
          If (feature 7 <= 1.5)
           If (feature 3 <= 7.414999961853027)
            Predict: 6843.5
           Else (feature 3 > 7.414999961853027)
            Predict: 6511.0
          Else (feature 7 > 1.5)
           Predict: 7214.0
         Else (feature 1 > 7304.0)
          If (feature 6 <= 2.5)
           If (feature 8 <= 3.5)
            Predict: 8544.176470588236
           Else (feature 8 > 3.5)
            Predict: 8037.0
          Else (feature 6 > 2.5)
           If (feature 3 <= 7.414999961853027)
            Predict: 8224.0
           Else (feature 3 > 7.414999961853027)
            Predict: 7539.333333333333
        Else (feature 1 > 8817.0)
         If (feature 8 <= 3.5)
          If (feature 6 <= 0.5)
           Predict: 10588.0
          Else (feature 6 > 0.5)
           If (feature 5 <= 4.4049999713897705)
            Predict: 10021.333333333334
           Else (feature 5 > 4.4049999713897705)
            Predict: 9800.746031746032
         Else (feature 8 > 3.5)
          If (feature 6 <= 2.5)
           If (feature 7 <= 1.5)
            Predict: 9845.739130434782
           Else (feature 7 > 1.5)
            Predict: 10129.5
          Else (feature 6 > 2.5)
           If (feature 8 <= 4.5)
            Predict: 10074.5
           Else (feature 8 > 4.5)
            Predict: 10580.285714285714
      Else (feature 5 > 4.575000047683716)
       If (feature 6 <= 3.5)
        If (feature 5 <= 4.884999990463257)
         If (feature 1 <= 8817.0)
          If (feature 6 <= 0.5)
           If (feature 1 <= 7304.0)
            Predict: 6819.0
           Else (feature 1 > 7304.0)
            Predict: 8125.0
          Else (feature 6 > 0.5)
           If (feature 0 <= 60.04999923706055)
            Predict: 8055.425
           Else (feature 0 > 60.04999923706055)
            Predict: 8778.0
         Else (feature 1 > 8817.0)
          If (feature 7 <= 1.5)
           If (feature 2 <= 1.524999976158142)
            Predict: 9379.555555555555
           Else (feature 2 > 1.524999976158142)
            Predict: 9912.333333333334
          Else (feature 7 > 1.5)
           If (feature 0 <= 60.04999923706055)
            Predict: 10365.038461538461
           Else (feature 0 > 60.04999923706055)
            Predict: 9314.0
        Else (feature 5 > 4.884999990463257)
         If (feature 8 <= 3.5)
          If (feature 2 <= 1.9800000190734863)
           Predict: 10798.0
          Else (feature 2 > 1.9800000190734863)
           Predict: 10640.0
         Else (feature 8 > 3.5)
          Predict: 10090.0
       Else (feature 6 > 3.5)
        If (feature 0 <= 54.95000076293945)
         If (feature 1 <= 8817.0)
          Predict: 8687.0
         Else (feature 1 > 8817.0)
          Predict: 8943.0
        Else (feature 0 > 54.95000076293945)
         If (feature 2 <= 1.524999976158142)
          If (feature 0 <= 56.04999923706055)
           If (feature 8 <= 3.5)
            Predict: 9533.5
           Else (feature 8 > 3.5)
            Predict: 8092.0
          Else (feature 0 > 56.04999923706055)
           If (feature 7 <= 1.5)
            Predict: 10561.2
           Else (feature 7 > 1.5)
            Predict: 10553.0
         Else (feature 2 > 1.524999976158142)
          If (feature 4 <= 7.4049999713897705)
           Predict: 9691.2
          Else (feature 4 > 7.4049999713897705)
           If (feature 7 <= 1.5)
            Predict: 9982.857142857143
           Else (feature 7 > 1.5)
            Predict: 8970.333333333334
    Else (feature 1 > 10901.0)
     If (feature 4 <= 7.9100000858306885)
      If (feature 2 <= 1.4049999713897705)
       If (feature 1 <= 14203.5)
        If (feature 2 <= 1.0449999570846558)
         If (feature 7 <= 5.5)
          If (feature 0 <= 56.04999923706055)
           If (feature 5 <= 3.9950000047683716)
            Predict: 11214.0
           Else (feature 5 > 3.9950000047683716)
            Predict: 10973.5
          Else (feature 0 > 56.04999923706055)
           If (feature 0 <= 57.04999923706055)
            Predict: 12445.0
           Else (feature 0 > 57.04999923706055)
            Predict: 11328.555555555555
         Else (feature 7 > 5.5)
          If (feature 5 <= 3.9950000047683716)
           If (feature 4 <= 6.454999923706055)
            Predict: 11442.0
           Else (feature 4 > 6.454999923706055)
            Predict: 11196.0
          Else (feature 5 > 3.9950000047683716)
           If (feature 6 <= 3.5)
            Predict: 10927.0
           Else (feature 6 > 3.5)
            Predict: 11237.25
        Else (feature 2 > 1.0449999570846558)
         If (feature 8 <= 4.5)
          If (feature 0 <= 59.10000038146973)
           If (feature 4 <= 6.855000019073486)
            Predict: 11262.714285714286
           Else (feature 4 > 6.855000019073486)
            Predict: 11512.648648648648
          Else (feature 0 > 59.10000038146973)
           If (feature 5 <= 4.234999895095825)
            Predict: 11019.0
           Else (feature 5 > 4.234999895095825)
            Predict: 12481.75
         Else (feature 8 > 4.5)
          If (feature 7 <= 4.5)
           If (feature 7 <= 3.5)
            Predict: 11760.67441860465
           Else (feature 7 > 3.5)
            Predict: 12081.936507936507
          Else (feature 7 > 4.5)
           If (feature 3 <= 6.855000019073486)
            Predict: 12148.581818181818
           Else (feature 3 > 6.855000019073486)
            Predict: 13231.375
       Else (feature 1 > 14203.5)
        If (feature 8 <= 6.5)
         If (feature 8 <= 5.5)
          If (feature 0 <= 58.04999923706055)
           If (feature 3 <= 6.855000019073486)
            Predict: 14414.0
           Else (feature 3 > 6.855000019073486)
            Predict: 14724.8
          Else (feature 0 > 58.04999923706055)
           Predict: 15802.0
         Else (feature 8 > 5.5)
          If (feature 5 <= 4.125)
           If (feature 0 <= 56.04999923706055)
            Predict: 14498.0
           Else (feature 0 > 56.04999923706055)
            Predict: 14525.0
          Else (feature 5 > 4.125)
           If (feature 5 <= 4.4049999713897705)
            Predict: 15712.583333333334
           Else (feature 5 > 4.4049999713897705)
            Predict: 15968.0
        Else (feature 8 > 6.5)
         If (feature 0 <= 57.04999923706055)
          If (feature 7 <= 4.5)
           If (feature 0 <= 56.04999923706055)
            Predict: 15437.0
           Else (feature 0 > 56.04999923706055)
            Predict: 14345.142857142857
          Else (feature 7 > 4.5)
           If (feature 0 <= 56.04999923706055)
            Predict: 16346.5
           Else (feature 0 > 56.04999923706055)
            Predict: 17584.5
         Else (feature 0 > 57.04999923706055)
          If (feature 6 <= 3.5)
           If (feature 0 <= 58.04999923706055)
            Predict: 17971.0
           Else (feature 0 > 58.04999923706055)
            Predict: 17240.0
          Else (feature 6 > 3.5)
           Predict: 15193.0
      Else (feature 2 > 1.4049999713897705)
       If (feature 4 <= 7.4049999713897705)
        If (feature 0 <= 58.04999923706055)
         If (feature 4 <= 7.134999990463257)
          If (feature 5 <= 4.575000047683716)
           If (feature 7 <= 5.5)
            Predict: 11653.75
           Else (feature 7 > 5.5)
            Predict: 10967.0
          Else (feature 5 > 4.575000047683716)
           If (feature 7 <= 5.5)
            Predict: 13070.333333333334
           Else (feature 7 > 5.5)
            Predict: 15152.0
         Else (feature 4 > 7.134999990463257)
          If (feature 1 <= 14203.5)
           If (feature 3 <= 7.414999961853027)
            Predict: 12663.558333333332
           Else (feature 3 > 7.414999961853027)
            Predict: 12011.894736842105
          Else (feature 1 > 14203.5)
           If (feature 8 <= 3.5)
            Predict: 15083.744680851063
           Else (feature 8 > 3.5)
            Predict: 15872.265625
        Else (feature 0 > 58.04999923706055)
         If (feature 2 <= 1.524999976158142)
          If (feature 0 <= 60.04999923706055)
           If (feature 7 <= 2.5)
            Predict: 11698.535714285714
           Else (feature 7 > 2.5)
            Predict: 14093.629032258064
          Else (feature 0 > 60.04999923706055)
           If (feature 1 <= 14203.5)
            Predict: 12850.375
           Else (feature 1 > 14203.5)
            Predict: 15701.8
         Else (feature 2 > 1.524999976158142)
          If (feature 1 <= 14203.5)
           If (feature 6 <= 2.5)
            Predict: 11374.333333333334
           Else (feature 6 > 2.5)
            Predict: 11827.0
          Else (feature 1 > 14203.5)
           If (feature 8 <= 3.5)
            Predict: 15249.0
           Else (feature 8 > 3.5)
            Predict: 14451.0
       Else (feature 4 > 7.4049999713897705)
        If (feature 7 <= 2.5)
         If (feature 6 <= 0.5)
          If (feature 5 <= 4.575000047683716)
           Predict: 11263.0
          Else (feature 5 > 4.575000047683716)
           If (feature 7 <= 0.5)
            Predict: 13399.0
           Else (feature 7 > 0.5)
            Predict: 15117.0
         Else (feature 6 > 0.5)
          If (feature 1 <= 14203.5)
           If (feature 2 <= 1.524999976158142)
            Predict: 11641.954545454546
           Else (feature 2 > 1.524999976158142)
            Predict: 12342.086021505376
          Else (feature 1 > 14203.5)
           If (feature 8 <= 3.5)
            Predict: 15858.636363636364
           Else (feature 8 > 3.5)
            Predict: 14949.5
        Else (feature 7 > 2.5)
         If (feature 1 <= 14203.5)
          If (feature 6 <= 3.5)
           If (feature 8 <= 3.5)
            Predict: 12824.325
           Else (feature 8 > 3.5)
            Predict: 13310.266666666666
          Else (feature 6 > 3.5)
           If (feature 0 <= 54.95000076293945)
            Predict: 13017.666666666666
           Else (feature 0 > 54.95000076293945)
            Predict: 13569.684210526315
         Else (feature 1 > 14203.5)
          If (feature 5 <= 4.575000047683716)
           If (feature 7 <= 5.5)
            Predict: 15506.714285714286
           Else (feature 7 > 5.5)
            Predict: 16657.777777777777
          Else (feature 5 > 4.575000047683716)
           If (feature 7 <= 3.5)
            Predict: 16365.46590909091
           Else (feature 7 > 3.5)
            Predict: 16731.160714285714
     Else (feature 4 > 7.9100000858306885)
      If (feature 1 <= 14203.5)
       If (feature 8 <= 3.5)
        Predict: 13328.346153846154
       Else (feature 8 > 3.5)
        If (feature 0 <= 53.95000076293945)
         Predict: 13203.0
        Else (feature 0 > 53.95000076293945)
         If (feature 0 <= 58.04999923706055)
          Predict: 11970.0
         Else (feature 0 > 58.04999923706055)
          If (feature 6 <= 2.5)
           If (feature 2 <= 1.9800000190734863)
            Predict: 12923.0
           Else (feature 2 > 1.9800000190734863)
            Predict: 12845.0
          Else (feature 6 > 2.5)
           Predict: 12963.0
      Else (feature 1 > 14203.5)
       If (feature 0 <= 57.04999923706055)
        If (feature 7 <= 1.5)
         If (feature 0 <= 54.95000076293945)
          Predict: 15939.375
         Else (feature 0 > 54.95000076293945)
          Predict: 16826.955555555556
        Else (feature 7 > 1.5)
         If (feature 3 <= 7.924999952316284)
          Predict: 18640.0
         Else (feature 3 > 7.924999952316284)
          Predict: 17763.0
       Else (feature 0 > 57.04999923706055)
        If (feature 7 <= 0.5)
         If (feature 6 <= 3.5)
          If (feature 0 <= 60.04999923706055)
           If (feature 6 <= 0.5)
            Predict: 14750.0
           Else (feature 6 > 0.5)
            Predict: 15730.0
          Else (feature 0 > 60.04999923706055)
           Predict: 17451.0
         Else (feature 6 > 3.5)
          If (feature 0 <= 58.04999923706055)
           If (feature 8 <= 4.5)
            Predict: 15450.0
           Else (feature 8 > 4.5)
            Predict: 17327.0
          Else (feature 0 > 58.04999923706055)
           Predict: 16499.666666666668
        Else (feature 7 > 0.5)
         If (feature 0 <= 60.04999923706055)
          If (feature 6 <= 1.5)
           If (feature 0 <= 59.10000038146973)
            Predict: 18023.0
           Else (feature 0 > 59.10000038146973)
            Predict: 18561.0
          Else (feature 6 > 1.5)
           If (feature 5 <= 4.884999990463257)
            Predict: 16694.0
           Else (feature 5 > 4.884999990463257)
            Predict: 17023.897058823528
         Else (feature 0 > 60.04999923706055)
          If (feature 0 <= 61.95000076293945)
           If (feature 6 <= 3.5)
            Predict: 16417.4
           Else (feature 6 > 3.5)
            Predict: 16512.0
          Else (feature 0 > 61.95000076293945)
           If (feature 6 <= 0.5)
            Predict: 17713.0
           Else (feature 6 > 0.5)
            Predict: 16055.285714285714

 ```

Your explanation here:

1) What specific predictor variable is being split and what is the value that determines the left / right split.
<br><br> Answer:
<br>feature 1 price: 1330.5	<br>feature 2 carat: 0.32500000298023224 <br>feature 4 table: 6.375 <br> feature 7 z: 2.5 <br>feature 8 cut: 5.5 

<br> Describe why the top level predictor changes from run to run.

 Answer: <br>
Since random forest model randomly selects a sample of features to train a tree for each run, it is possible that the features that are selected to train a top-level tree can be different from run to run.



# Question 6 (5 pts)
Describe if the random forest model MSE score was better or worse than the MSE score from you best model in homework 3.  Include both scores in your description.

Your improvement explanation here:  

Traced back my training and testing MSE score from HW3, I obtained 1487157.5078160448 and 1453693.4824793225 for each from the linear regression model. On the other hand, we obtained a training MSE score of 95944.05442283268 and a testing MSE score of 109094.2310196216. It is obvious that random forest regressor has a significantly lower MSE than normal linear regression model.

One of the main reasons that the MSE improvement is due to a huge reduction of variance from random forest regression model. We randomly selected predictors/ features out of all features to train each model. After we trained all small models (a.k.a random trees) based on randomly selected features, we fed training data to each model to generate multiple recommendations/predictions. To come up with a final recommendation, the random forest regressor took a majority vote from all the generated predictions from all learner models. Through this way, variance from trainning model design is significantly improved, which, in terms, lower the MSE.

# Question 7 (5 pts)
Set the `enable_grid_search` Boolean variable to False in the grading cell at the top of this notebook.  Perform a __Runtime -> factory reset__, __Runtime -> Run all__ test to verify there are no runtime errors.  Leave the `enable_grid_search` variable set to False and turn in your assignment.  This is the kind of thing you should be doing before you turn in every assignment. Remember this for future classes and when you get a job in industry.  This question will be graded as all or nothing.  You ether set the Boolean correct or not.  Additional points will be deducted elsewhere for runtime errors.

# Extra Credit (10 pts)
This homework was intended to take less time to complete and be about half the effort of previous assignments.  This doesn't allow us to explore GBT or deep learning.  

For extra credit, train a GBT or Deep Learning model using a grid search.  Protect the grid search inside the if enable_grid_search statement in the first code cell below.  You are free to use K-Fold cross validation if you wish.  The spark documentation for GBM can be found [here](https://spark.apache.org/docs/latest/ml-classification-regression.html#gradient-boosted-tree-classifier).  The spark documentation for deep learning can be found [here](https://spark.apache.org/docs/latest/ml-classification-regression.html#multilayer-perceptron-classifier)

In the second code cell below, hard code the best model parameters as determined by the grid search in a new pipeline named `best_pipe_2`.  Train and test `best_pipe_2` and save your resulting test MSE in a variable.  Do not use K-Fold cross validation when training best_pipe_2.  

In the third code cell below, create a pandas data frame named `compare_1_df` which contains 2 columns: Model and MSE.  Populate the Model column with model names: LR, RF, GBT or DL.  Populate the score column with the linear regression, random forest, and gradient boosted tree or deep learning test MSE scores. The linear regression score is from homework 3. The random forest score is from the random forest model above.  The GBT or Deep Learning score is from this extra credit problem.  Sort compare_1_df such that the best score is in the first row of the data frame. 

To get full credit, your GBT or deep learning solution should produce a score as good or better than the random forest score above.  In addition, the same rules as above apply where all of your grid search code shall be protected by the enable_grid_search Boolean.  Code that produces a runtime error when enable_grid_search is set to False will get 0 credit.

In [11]:
# Your GBT / Deep Learning grid search code here
from pyspark.ml.regression import GBTRegressor
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator

if enable_grid_search:
  diamonds_df_xformed = diamonds_df_xformed.select([diamonds_df_xformed.columns[3]] + diamonds_df_xformed.columns[:3] + diamonds_df_xformed.columns[4:])
  training_df, testing_df = diamonds_df_xformed.randomSplit([0.7, 0.3])

  # Establish pipeline objects
  va = VectorAssembler().setInputCols(training_df.columns[2:]).setOutputCol('features')
  fi = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4)
  gbt = GBTRegressor(featuresCol = 'indexedFeatures', labelCol="price")
  gbt_pipeline = Pipeline(stages=[va,fi,gbt])
  mse_evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mse")
  
  # Grid Build
  paramGrid_gbt = ParamGridBuilder().addGrid(gbt.maxBins, [35,40,45]).addGrid(gbt.maxDepth, [3,5,8]).build()
  cv_gbt = CrossValidator(estimator=gbt_pipeline, estimatorParamMaps=paramGrid_gbt, evaluator=mse_evaluator, numFolds=3)
  
  # Create search grid for training
  cv_fit = cv_gbt.fit(training_df)
  cv_predict = cv_fit.transform(testing_df)
  cv_mse = mse_evaluator.evaluate(cv_predict)
  print("MSE is:", cv_mse)

  grid_param_dict = cv_fit.bestModel.stages[-1].extractParamMap()
  
  result = {}
  for x,y in grid_param_dict.items():
    result[x.name]= y

  best_maxdepth = result["maxDepth"]
  best_maxbins = result["maxBins"]

  print('Best Param (maxdepth):{:.3f}'.format(best_maxdepth))
  print('Best Param (maxbins):{:.3f}'.format(best_maxbins))  

  pass

In [12]:
# your hard coded parameter best model code here
from pyspark.ml.regression import GBTRegressor
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator

training_df, testing_df = diamonds_df_xformed.randomSplit([0.7, 0.3])

# Establish pipeline objects
va = VectorAssembler().setInputCols(training_df.columns[1:]).setOutputCol('features')
fi = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4)
gbt = GBTRegressor().setFeaturesCol('indexedFeatures').setLabelCol("price").setMaxDepth(8).setMaxBins(45)
mse_evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mse")

# train model and predict
best_pipe_2 = Pipeline(stages=[va, fi, gbt]).fit(training_df)
cv_train_predict = best_pipe_2.transform(training_df)
cv_test_predict = best_pipe_2.transform(testing_df)

# Evaluate model with MSE metric
cv_train_mse = mse_evaluator.evaluate(cv_train_predict)
cv_test_mse = mse_evaluator.evaluate(cv_test_predict)

print("Training MSE is:", cv_train_mse)
print("Testing MSE is:", cv_test_mse)

Training MSE is: 20030.865415788583
Testing MSE is: 39868.05902920976


In [13]:
# Create compare_1_df
import pandas as pd

model_data = {'Model': ['LR','RF','GBT'], 'MSE': [1453693.4824793225 ,109094.2310196216,37358.47397678875]}
compare_1_df = pd.DataFrame(model_data, columns = ['Model', 'MSE']).sort_values('MSE', ascending=True)

In [14]:
# Grading cell do not modify
display(compare_1_df)

,Model,MSE
2,GBT,3.735847e+04
1,RF,1.090942e+05
0,LR,1.453693e+06
